# Downloading The Data

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

import scipy.stats as stats
import math

import sklearn.linear_model

In [ ]:
# Example cache directory path, it determines where downloaded data will be stored
base_directory = 'C:\\Users\\claym\\OneDrive\\Documents\\OSU\\uPNC\\Project\\Data'

out_directory = os.path.join(base_directory, 'Plots and output')

data_directory = os.path.join(base_directory, 'Manifest')

manifest_path = os.path.join(data_directory, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path,timeout=12000)

In [ ]:
sessionID_1 = 715093703
sessionID_2 = 739448407
sessionID_3 = 732592105
session_train = cache.get_session_data(sessionID_1)
session_test = cache.get_session_data(sessionID_1)

In [ ]:
probe_id = session_test.probes[session_test.probes.description == 'probeC'].index.values[0]

lfp = session_train.get_lfp(probe_id)

lfp_VISp_test = lfp.sel(channel = lfp.channel[54:67])


In [ ]:
probe_id = session.probes[session.probes.description == 'probeE'].index.values[0]

lfp = session.get_lfp(probe_id)

# lfp_VISrl = lfp.sel(channel = lfp.channel[64:85])

lfp_thalamus = lfp.sel(channel = lfp.channel[42:63])

# del globals()[lfp]

In [ ]:
probe_id = session.probes[session.probes.description == 'probeD'].index.values[0]

lfp = session.get_lfp(probe_id)

# lfp_LGd = lfp.sel(channel = lfp.channel[:20])

lfp_VISl = lfp.sel(channel = lfp.channel[60:80])

In [ ]:
probe_id = session_train.probes[session_train.probes.description == 'probeC'].index.values[0]

lfp = session_train.get_lfp(probe_id)

# lfp_VISp = lfp.sel(channel = lfp.channel[60:80])

lfp_VISp_train = lfp.sel(channel = lfp.channel[63:76])

# lfp_LP = lfp.sel(channel = lfp.channel[5:25])


In [ ]:
probe_id = session.probes[session.probes.description == 'probeB'].index.values[0]

lfp = session.get_lfp(probe_id)

lfp_VISpm = lfp.sel(channel = lfp.channel[66:81])

In [ ]:
probe_id = session.probes[session.probes.description == 'probeA'].index.values[0]

lfp = session.get_lfp(probe_id)

lfp_VISam = lfp.sel(channel = lfp.channel[60:80])

 # Probe C

The motivation in investigating Probe C is to isolate V1 (VISp)

## NWBWIDGETS

In [ ]:
from pynwb import NWBHDF5IO
import allensdk.brain_observatory.ecephys.nwb

from nwbwidgets import nwb2widget
fpath = '/Users/willi/OneDrive/Documents/Computational Neuroscience/Project/ecephys_guides/session_715093703/probe_810755805_lfp-004.nwb'
#this is probe C which passes through V1, specific interest in 62-82 or VISp. Clay mentions 135 and 315 are orientations of interest
nwb = NWBHDF5IO(fpath, 'r').read()


#from nwbwidgets.allen import AllenRasterWidget

#AllenRasterWidget(nwb.units)

In [ ]:
nwb2widget(nwb)


In [ ]:
from pynwb import NWBHDF5IO
import allensdk.brain_observatory.ecephys.nwb
import ndx_icephys_meta
from nwbwidgets import nwb2widget
fpath = '/Users/willi/downloads/session_715093703/probe_810755801_lfp.nwb'
#this is probe C which passes through V1, specific interest in 62-82 or VISp. Clay mentions 135 and 315 are orientations of interest
nwb = NWBHDF5IO(fpath, 'r').read()
nwb2widget(nwb)

In [ ]:
#from nwbwidgets.allen import AllenRasterWidget
#
#AllenRasterWidget(nwb.units)

# LFP Exploration

Below is loading in the specific probe of interest and selecting the region of interest VISp

In [ ]:
probe_id = session.probes[session.probes.description == 'probeC'].index.values[0]
##from clay print(session.probes[session.probes.description == 'probeC'].index.values)
lfp = session.get_lfp(probe_id)
lfp_VISp = lfp.sel(channel = lfp.channel[62:84])

In [ ]:
lfp

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable

device = torch.device("cuda:0")
torch.cuda.set_device(device)

import torchvision.models as models

# io utils
from pytorch3d.io import load_obj

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    Textures,
    look_at_view_transform,
    OpenGLPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    BlendParams
)

import os
import numpy as np
from PIL import Image
print("not the import")



# function loading a 3d obj and texture using pytorch3d
def load_obj_and_texture_img_as_mesh(f_obj, f_tex, device=None, texture_requires_grad=False):
  verts, faces, aux = load_obj(f_obj, load_textures=False)
  verts = verts.to(device)
  
  if aux is None or aux.verts_uvs is None:
    raise ValueError(f"{f_obj} and {f_tex} loaded incorrectly")
  
  verts_uvs = aux.verts_uvs[None, ...].to(device)  # (1, V, 2)
  faces_uvs = faces.textures_idx[None, ...].to(device)  # (1, F, 3)
  image = torch.as_tensor(np.array(Image.open(f_tex).convert("RGB")) / 255.0, dtype=torch.float32).to(device)[None]
  tex = Textures(verts_uvs=verts_uvs, faces_uvs=faces_uvs, maps=image)
  
  mesh = Meshes(verts=[verts], faces=[faces.verts_idx.to(device)], textures=tex)
  
  if texture_requires_grad:
    mesh.textures._maps_padded.requires_grad = True
  
  return mesh


# Data preprocessing for inception_v3
def process_data(device, data, input_size):
    data = torch.squeeze(data)
    data = (data - data.min()) / (data.max() - data.min())  # rescale to [0, 1]
    mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
    std = torch.tensor([0.229, 0.224, 0.225]).to(device)
    data = (data - mean[: ,None, None]) / std[: ,None, None]  # normalise with tensors
    data = torch.unsqueeze(data,0)
    data = F.interpolate(data, (input_size, input_size))
    return data

# # multiview setting
PARAMETER_PATH = "/data03/home/cwashington/PyTorch3D/data_stop_new/tv_nps_ce_eykholt_goodplacement/test_set_params_short.txt"
param_file = open(PARAMETER_PATH, 'r')
testing_params = []
for line in param_file:
  testing_params.append(line.split(','))
  testing_params[len(testing_params)-1] = list(map(float, testing_params[len(testing_params)-1]))
param_file.close()

image_size = 1024  # for high quality
faces_per_pixel = 10


num_renders_per_iteration = len(testing_params)
print(num_renders_per_iteration)

##LOCAL ATTACK ATTEMPT (CE ONLY)
import random

#phase 1: masking image effectively

#load image
DATA_DIR = "/data03/home/cwashington/PyTorch3D/data_stop_new"
TEXTURE_DIR_ROOT = os.path.join(DATA_DIR, "tv_nps_ce_eykholt_goodplacement")


#phase 2: perform attack and eliminate gradient in masked area

# # Set up Renderer
raster_settings = RasterizationSettings(image_size=image_size,
                                        blur_radius=0.0,
                                        faces_per_pixel=faces_per_pixel,
                                        bin_size=None,
                                        max_faces_per_bin=None
)

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=None,
        raster_settings=raster_settings,
    ),
    shader=SoftPhongShader(
        device=device,
        cameras=None,
        lights=None,
    )
)

target = Variable(torch.LongTensor([919]), requires_grad=False)
target = target.to(device)


# # load 3d objects and texture

obj_path = os.path.join(DATA_DIR, "stopsign.obj")
max_texture_number = 149
save_success_ratios = [None]*(max_texture_number)



def initialize_model(model_name, use_pretrained=True):
        # Initialize these variables which will be set in this if statement. Each of these
        #   variables is model specific.
        model_ft = None
        input_size = 0

        if model_name == "resnet18":
            """ Resnet18
            """
            model_ft = models.resnet18(pretrained=use_pretrained)
            input_size = 224

        elif model_name == "alexnet":
            """ Alexnet
            """
            model_ft = models.alexnet(pretrained=use_pretrained)
            input_size = 224

        elif model_name == "squeezenet1_0":
            """ Squeezenet
            """
            model_ft = models.squeezenet1_0(pretrained=use_pretrained)
            input_size = 224

        elif model_name == "vgg16":
            """ VGG16
            """
            model_ft = models.vgg16(pretrained=use_pretrained)
            input_size = 224

        elif model_name == "densenet161":
            """ Densenet
            """
            model_ft = models.densenet161(pretrained=use_pretrained)
            input_size = 224

        elif model_name == "inception_v3":
            """ Inception v3 
            Be careful, expects (299,299) sized images and has auxiliary output
            """
            model_ft = models.inception_v3(pretrained=use_pretrained)
            input_size = 299
        else:
            print("Invalid model name, exiting...")
            exit()

        return model_ft, input_size
    
PRETRAINED_NETMODEL_NAMES = ['resnet18','alexnet','squeezenet1_0','vgg16','densenet161','inception_v3']
PRETRAINED_NETMODEL_NAMES_1 = ['resnet18','alexnet','squeezenet1_0']
PRETRAINED_NETMODEL_NAMES_2 = ['vgg16','densenet161','inception_v3']

for PRETRAINED_NETMODEL_NAME in PRETRAINED_NETMODEL_NAMES_1:

    print(PRETRAINED_NETMODEL_NAME)

    


    # Initialize the model for this run
    model_ft, input_size = initialize_model(PRETRAINED_NETMODEL_NAME)

    model = model_ft.to(device)

    model.eval()

    ##################################################
    
    for PERTURBATION_SOURCE in PRETRAINED_NETMODEL_NAMES:

        texture_path = os.path.join(TEXTURE_DIR_ROOT, PERTURBATION_SOURCE, "perturbed_texture_12.png")
        mesh = load_obj_and_texture_img_as_mesh(obj_path, texture_path, device=device)

        count = 0
        close_count = 0
        med_count = 0
        far_count = 0

        for i in range(len(testing_params)):

            params = testing_params[i]

            ambient_color = params[0]
            dist = params[1]
            elev = params[2]
            azim = params[3]    

            materials = Materials(device=device, ambient_color=[[ambient_color*1] * 3], specular_color=[[ambient_color*.3]*3], shininess = 32)


            # get look at, openGL perspec camera according to current parameters
            R, T = look_at_view_transform(dist, elev, azim)
            cameras = OpenGLPerspectiveCameras(device=device, R=R, T=T)
            location = cameras.get_camera_center().detach().cpu().numpy()
            lights = PointLights(device=device, location = location, ambient_color=((ambient_color, ambient_color, ambient_color),), diffuse_color=((ambient_color*.9, ambient_color*.9, ambient_color*.9),))

            # run neural renderer and process output for training targets
            images = renderer(mesh, cameras=cameras, materials=materials,  lights = lights)
            images = images[..., :3]

            # ### show the rendered images
            # imagesDos = images.clone()
            # imagesDos = imagesDos[0].cpu().detach().numpy()
            # plt.figure(figsize=(10, 10))
            # plt.imshow(imagesDos)
            # plt.grid("off");
            # plt.axis("off");
            # plt.show()

            images = images.permute(0, 3, 1, 2)

            processed_temp_images = process_data(device, images, input_size)

            output1 = model(processed_temp_images)

            init_pred = output1.max(1, keepdim=True)[1]  # get the index of the max log-probability

            if init_pred.item() != target.data.item():
              count = count + 1

        success_ratio = count/num_renders_per_iteration
        print("attacking model: {} victim model: {} success rate: {}".format(PERTURBATION_SOURCE,PRETRAINED_NETMODEL_NAME,str(success_ratio)))




Next we desire to isolate time slices and visualize them through a plot

In [ ]:
lfp_slice = lfp.sel(time=slice(1800,5410))

lfp_slice_VISp = lfp_slice.sel(channel=lfp_slice.channel[62:84])


In [ ]:
lfp_slice = lfp.sel(time=slice(1800,5410))

lfp_slice_VISp = lfp_slice.sel(channel=lfp_slice.channel[-25:-10]) ##playing around with the range for the matplotlib...
#...function further down


Obtaining multiple plots at once for lfp_slice_VISp with a for loop

In [ ]:
for i in range(15):
    plt.figure(figsize=(10,2))
    _ = plt.plot(lfp_slice_VISp.time, lfp_slice_VISp.sel(channel=lfp_slice_VISp.channel[i]))
    plt.xlabel('Time (s)')
    plt.ylabel('LFP (V)')

In [ ]:
lfp_slice_VISp

In [ ]:
plt.figure(figsize=(10,2))
_ = plt.plot(lfp_slice_VISp.time, lfp_slice_VISp)
plt.xlabel('Time (s)')
plt.ylabel('LFP (V)')

Alternatively, we can visualize lfp_slice_VISp using matplotlib's imshow method:
!!This is strangely lacking information so look at this again!!

In [ ]:
plt.figure(figsize=(16,16))
im = plt.imshow(lfp_slice_VISp.T,aspect='auto',origin='lower',vmin=-1e-3, vmax=1e-3)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')

Note that we've transposed the original array to place the time dimension along the x-axis. We've also configured the plot so that the origin of the array is in the lower-left, so that that channels closer to the probe tip are lower in the image.

A few things to note about this plot:

The units of the LFP are volts, so the color scale ranges from -1 to +1 mV

Even though there are 384 channels on the Neuropixels probe, there are only 95 channels in this plot. That's because only every 4th channel is included in the NWB file (resulting in 40 micron vertical spacing). In addition, the reference channels and channels far outside the brain have been removed.

The top of the plot is relatively flat. This corresponds to channels that are outside the brain. The LFP channels are originally referenced to a ground wire embedded in the ACSF/agarose mixture about cortex. Before NWB packaging, the LFP data is digitally referenced to the channels outside the brain, to remove noise that's shared across the whole probe.
There's a large increase in LFP power toward the middle of the probe, which corresponds to channels in hippocampus.

Let's do some additional data selection to look at just the visual recordings

In [ ]:
channel_ids = session.channels[(session.channels.probe_id == probe_id) & \
                 (session.channels.ecephys_structure_acronym.isin(['VISp']))].index.values

lfp_slice2 = lfp_slice.sel(channel=slice(np.min(channel_ids), np.max(channel_ids)))

plt.figure(figsize=(8,4))
im = plt.imshow(lfp_slice2.T,aspect='auto',origin='lower',vmin=-1e-3, vmax=1e-3)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number') ## Sample number is time: think of it like each time point is a sample
_ = plt.ylabel('Channel index')

What follows next is a sequence of code cells from Allen/ Written by Clay, edited by myself which appear to be attempts to isolate the stimulus presentation for the probe. The code is fairly straightforward and will be used for work on Probe D Analysis

# transferability experiment

In [ ]:
import random
import statistics
from sklearn.model_selection import train_test_split

flashes_presentation_table_train = session_train.stimulus_presentations[session_train.stimulus_presentations.stimulus_name == 'flashes']
flashes_presentation_times_train = flashes_presentation_table_train.start_time.values
flashes_presentation_ids_train = flashes_presentation_table_train.index.values

flashes_presentation_table_test = session_test.stimulus_presentations[session_test.stimulus_presentations.stimulus_name == 'flashes']
flashes_presentation_times_test = flashes_presentation_table_test.start_time.values
flashes_presentation_ids_test = flashes_presentation_table_test.index.values


#TRAIN MODEL


#CREATING PLOTS WITH NCV

# LINEAR CLASSIFIER

#specifically works for flashes
def lr_transferability_train(flashes_presentation_table, lfp_data, window_size, end_time, time_res):
    classifier_accuracies = []
    models = []
    
    window_start_times = []
    start_time = 0
    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    print(window_start_times)

    next_empty_column = 0
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
    
 


    #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
            b = aligned_lfp_1,
            axis = 1,
            equal_var = False,
            nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)


        x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
        
        
        model = sklearn.linear_model.LogisticRegression(solver='liblinear')

        penalty = ['l1','l2']
        C = [10,100,1000]
        hyperparameters = dict(C=C,penalty=penalty)

        clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
        best_model_LR = clf.fit(x_train,y_train)
        models.append(best_model_LR)
        classifier_accuracies.append(best_model_LR.score(x_test,y_test))

        
    return window_start_times, models, classifier_accuracies

def lr_transferability_test(flashes_presentation_table, lfp_data, window_size, end_time, time_res, models):
    classifier_accuracies = []
    model_idx = 0
    
    window_start_times = []
    start_time = 0
    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    print(window_start_times)

    next_empty_column = 0
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
    
 


    #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
            b = aligned_lfp_1,
            axis = 1,
            equal_var = False,
            nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)


#         x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=1, stratify=labels)
        
        
        best_model_LR = models[model_idx]
        classifier_accuracies.append(best_model_LR.score(flashes_data_combined,labels))

        model_idx += 1
        
    return classifier_accuracies

#-----------------------------------------------------------------------------------------

# starts, models, train_accuracies = lr_transferability_train(flashes_presentation_table=flashes_presentation_table_train,
#                                                                          lfp_data=lfp_VISp_train, 
#                                                                          window_size=.01, 
#                                                                          end_time=.5, 
#                                                                          time_res=1/100)
# print("train complete")

# test_accuracies = lr_transferability_test(flashes_presentation_table=flashes_presentation_table_test,
#                                                                          lfp_data=lfp_VISp_train, 
#                                                                          window_size=.01, 
#                                                                          end_time=.5, 
#                                                                          time_res=1/100,
#                                                                          models=models)

# for i in range(len(starts)):
#     print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, train_accuracies, label="Training mouse")
# _ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
im = plt.plot(starts, test_accuracies, label="New mouse")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Logistic regression classifer accuracy on mouse used for training and new mouse')
_ = plt.legend()

# plt.figure(figsize=(8,6))
# im = plt.plot(starts, errhat_ncv, label="mean")
# _ = plt.xlabel('Start of time window following stimulus onset (s)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
# _ = plt.legend()

# plt.figure(figsize=(8,6))
# im = plt.plot(starts, errhat_cv, label="mean")
# _ = plt.xlabel('Start of time window following stimulus onset (s)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
# _ = plt.legend()

# fig = plt.figure(figsize=(8,6))
# im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
# _ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
# _ = plt.ylabel("Electrode index")
# _ = plt.title("Linear Classifier Coefficients")
# _ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))










# 2x2x2 table and Kappa

In [ ]:
flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                      == 1.0]
flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                          == -1.0]

window_end = 0.5
time_res = 1/100
lfp_data = lfp_VISp

#for 1

presentation_times = flashes_presentation_table_1.start_time.values
presentation_ids = flashes_presentation_table_1.index.values

trial_window = np.arange(0, window_end, time_res)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_1 = ds['aligned_lfp']

#for -1

presentation_times = flashes_presentation_table_minus1.start_time.values
presentation_ids = flashes_presentation_table_minus1.index.values

trial_window = np.arange(0, window_end, time_res)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
       b = aligned_lfp_1,
       axis = 1,
       equal_var = False,
       nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


flashes_1_data = aligned_lfp_1.to_masked_array()
flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
flashes_1_data.mask = np.ma.nomask
flashes_minus1_data.mask = np.ma.nomask
flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
labels = []
for i in range(75):
    labels.append(1)
for i in range(75,150):
    labels.append(0)
    
NMC = 100

# GT/SVM/LR
a111=0
a110=0
a101=0
a100=0
a011=0
a010=0
a001=0
a000=0


SVM_out_total = []
LR_out_total = []
    
    
for a in range(NMC):

    # PERFORM  CLASSIFICATIONS


    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.5, stratify=labels)

    # TUNE HYPERPARAMETERS USING CROSS-VALIDATION

    model = sklearn.svm.SVC()

    kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    C = [10,100,1000]
    hyperparameters = dict(C=C,kernel=kernel)

    clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
    best_model_SVM = clf.fit(x_train,y_train)

    model = sklearn.linear_model.LogisticRegression(solver='liblinear')

    penalty = ['l1','l2']
    C = [10,100,1000]
    hyperparameters = dict(C=C,penalty=penalty)

    clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
    best_model_LR = clf.fit(x_train,y_train)



    ground_truth = y_test
    SVM_out = best_model_SVM.predict(x_test)
    LR_out = best_model_LR.predict(x_test)
    
    SVM_out_total.append(SVM_out)
    LR_out_total.append(LR_out)






    for i in range(len(ground_truth)):
        if ground_truth[i] == 1:
            if SVM_out[i] == 1:
                if LR_out[i] == 1:
                    a111 += 1
                else:
                    a110 += 1
            else:
                if LR_out[i] == 1:
                    a101 += 1
                else:
                    a100 += 1
        else:
            if SVM_out[i] == 1:
                if LR_out[i] == 1:
                    a011 += 1
                else:
                    a010 += 1
            else:
                if LR_out[i] == 1:
                    a001 += 1
                else:
                    a000 += 1
                    
tot = a111 + a110 + a101 + a100 + a011 + a010 + a001 + a000

print("111: {} \n110: {} \n101: {} \n100: {} \n011: {} \n010: {} \n001: {} \n000: {}".format(a111/tot,a110/tot,a101/tot,a100/tot,a011/tot,a010/tot,a001/tot,a000/tot))
kappa = sklearn.metrics.cohen_kappa_score(SVM_out,LR_out)
print("kappa:",kappa)

# # PLOTTING SLIDING TIME WINDOW-BASED NCV CIs FOR CLASSIFIER ACCURACY (4WILLIAM)

In [ ]:
# svm

def standard_CV(data,labels,K):
    scores = []
    rand_inds = [i for i in range(len(data))]
    random.shuffle(rand_inds)
    data_folds = []
    label_folds = []
    for i in range(K):
        fold_inds = rand_inds[round(len(rand_inds)*i/K):round(len(rand_inds)*(i+1)/K)]
        data_fold = []
        label_fold = []
        for ind in fold_inds:
            data_fold.append(data[ind])
            label_fold.append(labels[ind])
        data_folds.append(data_fold)
        label_folds.append(label_fold)
    for val_ind in range(K):
        val_data_fold = None
        val_label_fold = None 
        train_data_folds = []
        train_label_folds = []
        for fold_ind in range(K):
            if fold_ind == val_ind:
                val_data_fold = data_folds[fold_ind]
                val_label_fold = label_folds[fold_ind]
            else:
                train_data_folds.append(data_folds[fold_ind])
                train_label_folds.append(label_folds[fold_ind])
                
        val_data_fold = np.array(val_data_fold)
        val_label_fold = np.array(val_label_fold)
        train_data_folds = np.array(train_data_folds)
        train_label_folds = np.array(train_label_folds)


        train_data = train_data_folds.reshape(train_data_folds.shape[0]*train_data_folds.shape[1],train_data_folds.shape[2])
        train_labels = train_label_folds.flatten()       
        
        
        model = sklearn.svm.SVC()

        kernel = ['linear', 'sigmoid']
        C = [10,100,1000]
        hyperparameters = dict(C=C,kernel=kernel)

        clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)

        
        
        
        theta_hat = clf.fit(train_data,train_labels)
        mean_e_out = 1 - theta_hat.score(val_data_fold,val_label_fold)
        

        scores.append(mean_e_out)
    return scores

def inner_CV(data_folds,label_folds,K):
    e_in = []

    for val_ind in range(K-1):
        val_data_fold = None
        val_label_fold = None 
        train_data_folds = []
        train_label_folds = []
        for fold_ind in range(K-1):
            if fold_ind == val_ind:
                val_data_fold = data_folds[fold_ind]
                val_label_fold = label_folds[fold_ind]
            else:
                train_data_folds.append(data_folds[fold_ind])
                train_label_folds.append(label_folds[fold_ind])
                
        val_data_fold = np.array(val_data_fold)
        val_label_fold = np.array(val_label_fold)
        train_data_folds = np.array(train_data_folds)
        train_label_folds = np.array(train_label_folds)

                
        train_data = train_data_folds.reshape(train_data_folds.shape[0]*train_data_folds.shape[1],train_data_folds.shape[2])
        train_labels = train_label_folds.flatten()

        model = sklearn.svm.SVC()

        kernel = ['linear', 'sigmoid']
        C = [10,100,1000]
        hyperparameters = dict(C=C,kernel=kernel)

        clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
        theta_hat = clf.fit(train_data,train_labels)
        mean_e_out = 1 - theta_hat.score(val_data_fold,val_label_fold)
        num_1 = np.round_(mean_e_out*len(val_data_fold),2)
        num_0 = len(val_data_fold)-num_1
        fake_sample = []
        for i in range(int(num_1)):
            fake_sample.append(1)
        for i in range(int(num_0)):
            fake_sample.append(0)
        
        e_in.append(fake_sample)
    return e_in


def nested_CV(X,Y,R,K):
    es = []
    a_list = []
    b_list = []
    for r in range(R):
        #assign folds    CAN THROW IN STRATIFY USING FRACTIONS IN TRAIN TEST SPLIT WITH DECREASING DENOM
        rand_inds = [i for i in range(len(X))]
        random.shuffle(rand_inds)
        data_folds_for_rep = []
        label_folds_for_rep = []
        for i in range(K):
            fold_inds = rand_inds[round(len(rand_inds)*i/K):round(len(rand_inds)*(i+1)/K)]
            data_fold = []
            label_fold = []
            for ind in fold_inds:
                data_fold.append(X[ind])
                label_fold.append(Y[ind])
            data_folds_for_rep.append(data_fold)
            label_folds_for_rep.append(label_fold)

        #outer cv loop
        for val_ind in range(K):
            val_data_fold = None
            val_label_fold = None 
            train_data_folds = []
            train_label_folds = []
            for fold_ind in range(K):
                if fold_ind == val_ind:
                    val_data_fold = data_folds_for_rep[fold_ind]
                    val_label_fold = label_folds_for_rep[fold_ind]
                else:
                    train_data_folds.append(data_folds_for_rep[fold_ind])
                    train_label_folds.append(label_folds_for_rep[fold_ind])
                    
            val_data_fold = np.array(val_data_fold)
            val_label_fold = np.array(val_label_fold)
            train_data_folds = np.array(train_data_folds)
            train_label_folds = np.array(train_label_folds)
            
            e_in = inner_CV(train_data_folds,train_label_folds,K)

            
            train_data = train_data_folds.reshape(train_data_folds.shape[0]*train_data_folds.shape[1],train_data_folds.shape[2])
            train_labels = train_label_folds.flatten()
            
            model = sklearn.svm.SVC()

            kernel = ['linear', 'sigmoid']
            C = [10,100,1000]
            hyperparameters = dict(C=C,kernel=kernel)

            clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
            theta_hat = clf.fit(train_data,train_labels)
            
            mean_e_out = 1 - theta_hat.score(val_data_fold,val_label_fold)

            num_1 = np.round_(mean_e_out*len(val_data_fold),2)
            num_0 = len(val_data_fold)-num_1
            fake_sample = []
            for i in range(int(num_1)):
                fake_sample.append(1)
            for i in range(int(num_0)):
                fake_sample.append(0)
            var_e_out = statistics.variance(fake_sample)
            a_list.append((np.mean(e_in) - mean_e_out)**2)
            b_list.append(statistics.variance(fake_sample)/len(val_data_fold))
            es.append(e_in)
    

    MSEhat = np.mean(a_list) - np.mean(b_list)
    errhat_NCV = np.mean(es)
    return errhat_NCV, MSEhat

In [ ]:
import scipy.stats as st
import random

#CREATING PLOTS WITH NCV

# LINEAR CLASSIFIER

#specifically works for flashes
def classifier_accuracy_by_trial_window_ncv_lin(flashes_presentation_table, lfp_data, window_size, end_time, time_res, reps, k, alpha):
    classifier_accuracies_qalphadiv2 = []
    classifier_accuracies_means = []
    classifier_accuracies_q1minusalphadiv2 = []
    errhat_ncv_store = []
    errhat_cv_store = []
        
    
    window_start_times = []
    start_time = 0
    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    print(window_start_times)
    
    model_coefficients = np.zeros((len(lfp_data.channel),int(end_time/time_res)))
    print("model coefficients shape:",model_coefficients.shape)

    next_empty_column = 0
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
    
 


    #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
            b = aligned_lfp_1,
            axis = 1,
            equal_var = False,
            nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)



        errhat_ncv, MSEhat = nested_CV(X=flashes_data_combined, Y=labels, R=reps, K=k)
        
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
        


        errhat_cv = np.mean(standard_CV(flashes_data_combined,labels,k))
        
        
        errhat_ncv_store.append(errhat_ncv)
        errhat_cv_store.append(errhat_cv)
        
        biashat = (1+((k-2)/k))*(errhat_ncv-errhat_cv)
        point_estimate = errhat_ncv - biashat
        
        
        #CONSTRUCTION
        print("window:",window_start)
        print("k:",k,"MSEhat",MSEhat)
        print("NCV error estimate:",errhat_ncv,"point estimate:",point_estimate)
        
        #/CONSTRUCTION
        
        
        interval_radius = st.norm.ppf(1-alpha/2)*math.sqrt((k-1)/k)*math.sqrt(abs(MSEhat))     
        
    
#         coefficient_store = np.matrix(coefficient_store)
#         print(coefficient_store.shape)
#         coefficient_store = np.transpose(coefficient_store)
#         print(coefficient_store.shape)
#         coefficient_store = coefficient_store.mean(axis=1)
#         print(coefficient_store.shape)


        
    

        classifier_accuracies_qalphadiv2.append(min(1,1-(point_estimate - interval_radius)))
        classifier_accuracies_means.append(1-point_estimate)
        classifier_accuracies_q1minusalphadiv2.append(min(1,1-(point_estimate + interval_radius)))
        
    return window_start_times, classifier_accuracies_qalphadiv2, classifier_accuracies_means, classifier_accuracies_q1minusalphadiv2, errhat_ncv_store, errhat_cv_store

#-----------------------------------------------------------------------------------------

starts, ci_low, classifier_accuracies, ci_hi, errhat_ncv, errhat_cv = classifier_accuracy_by_trial_window_ncv_lin(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         reps=10, k=5,
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Logistic regression classifer accuracy with 95% CI from NCV')
_ = plt.legend()

# plt.figure(figsize=(8,6))
# im = plt.plot(starts, errhat_ncv, label="mean")
# _ = plt.xlabel('Start of time window following stimulus onset (s)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
# _ = plt.legend()

# plt.figure(figsize=(8,6))
# im = plt.plot(starts, errhat_cv, label="mean")
# _ = plt.xlabel('Start of time window following stimulus onset (s)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
# _ = plt.legend()

# fig = plt.figure(figsize=(8,6))
# im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
# _ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
# _ = plt.ylabel("Electrode index")
# _ = plt.title("Linear Classifier Coefficients")
# _ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

In [ ]:
import random
import statistics

flashes_presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'flashes']
flashes_presentation_times = flashes_presentation_table.start_time.values
flashes_presentation_ids = flashes_presentation_table.index.values

len(flashes_presentation_table)


In [ ]:
def standard_CV(data,labels,K):
    scores = []
    rand_inds = [i for i in range(len(data))]
    random.shuffle(rand_inds)
    data_folds = []
    label_folds = []
    for i in range(K):
        fold_inds = rand_inds[round(len(rand_inds)*i/K):round(len(rand_inds)*(i+1)/K)]
        data_fold = []
        label_fold = []
        for ind in fold_inds:
            data_fold.append(data[ind])
            label_fold.append(labels[ind])
        data_folds.append(data_fold)
        label_folds.append(label_fold)
    for val_ind in range(K):
        val_data_fold = None
        val_label_fold = None 
        train_data_folds = []
        train_label_folds = []
        for fold_ind in range(K):
            if fold_ind == val_ind:
                val_data_fold = data_folds[fold_ind]
                val_label_fold = label_folds[fold_ind]
            else:
                train_data_folds.append(data_folds[fold_ind])
                train_label_folds.append(label_folds[fold_ind])
                
        val_data_fold = np.array(val_data_fold)
        val_label_fold = np.array(val_label_fold)
        train_data_folds = np.array(train_data_folds)
        train_label_folds = np.array(train_label_folds)


        train_data = train_data_folds.reshape(train_data_folds.shape[0]*train_data_folds.shape[1],train_data_folds.shape[2])
        train_labels = train_label_folds.flatten()

        model = sklearn.linear_model.LogisticRegression(solver='liblinear')

        penalty = ['l1','l2']
        C = [10,100,1000]
        hyperparameters = dict(C=C,penalty=penalty)

        clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
        theta_hat = clf.fit(train_data,train_labels)
        mean_e_out = 1 - theta_hat.score(val_data_fold,val_label_fold)
        

        scores.append(mean_e_out)
    return scores

def inner_CV(data_folds,label_folds,K):
    e_in = []

    for val_ind in range(K-1):
        val_data_fold = None
        val_label_fold = None 
        train_data_folds = []
        train_label_folds = []
        for fold_ind in range(K-1):
            if fold_ind == val_ind:
                val_data_fold = data_folds[fold_ind]
                val_label_fold = label_folds[fold_ind]
            else:
                train_data_folds.append(data_folds[fold_ind])
                train_label_folds.append(label_folds[fold_ind])
                
        val_data_fold = np.array(val_data_fold)
        val_label_fold = np.array(val_label_fold)
        train_data_folds = np.array(train_data_folds)
        train_label_folds = np.array(train_label_folds)

                
        train_data = train_data_folds.reshape(train_data_folds.shape[0]*train_data_folds.shape[1],train_data_folds.shape[2])
        train_labels = train_label_folds.flatten()

        model = sklearn.linear_model.LogisticRegression(solver='liblinear')

        penalty = ['l1','l2']
        C = [10,100,1000]
        hyperparameters = dict(C=C,penalty=penalty)

        clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
        theta_hat = clf.fit(train_data,train_labels)
        mean_e_out = 1 - theta_hat.score(val_data_fold,val_label_fold)
        num_1 = np.round_(mean_e_out*len(val_data_fold),2)
        num_0 = len(val_data_fold)-num_1
        fake_sample = []
        for i in range(int(num_1)):
            fake_sample.append(1)
        for i in range(int(num_0)):
            fake_sample.append(0)
        
        e_in.append(fake_sample)
    return e_in


def nested_CV(X,Y,R,K):
    es = []
    a_list = []
    b_list = []
    for r in range(R):
        #assign folds    CAN THROW IN STRATIFY USING FRACTIONS IN TRAIN TEST SPLIT WITH DECREASING DENOM
        rand_inds = [i for i in range(len(X))]
        random.shuffle(rand_inds)
        data_folds_for_rep = []
        label_folds_for_rep = []
        for i in range(K):
            fold_inds = rand_inds[round(len(rand_inds)*i/K):round(len(rand_inds)*(i+1)/K)]
            data_fold = []
            label_fold = []
            for ind in fold_inds:
                data_fold.append(X[ind])
                label_fold.append(Y[ind])
            data_folds_for_rep.append(data_fold)
            label_folds_for_rep.append(label_fold)

        #outer cv loop
        for val_ind in range(K):
            val_data_fold = None
            val_label_fold = None 
            train_data_folds = []
            train_label_folds = []
            for fold_ind in range(K):
                if fold_ind == val_ind:
                    val_data_fold = data_folds_for_rep[fold_ind]
                    val_label_fold = label_folds_for_rep[fold_ind]
                else:
                    train_data_folds.append(data_folds_for_rep[fold_ind])
                    train_label_folds.append(label_folds_for_rep[fold_ind])
                    
            val_data_fold = np.array(val_data_fold)
            val_label_fold = np.array(val_label_fold)
            train_data_folds = np.array(train_data_folds)
            train_label_folds = np.array(train_label_folds)
            
            e_in = inner_CV(train_data_folds,train_label_folds,K)

            
            train_data = train_data_folds.reshape(train_data_folds.shape[0]*train_data_folds.shape[1],train_data_folds.shape[2])
            train_labels = train_label_folds.flatten()
            
            model = sklearn.linear_model.LogisticRegression(solver='liblinear')

            penalty = ['l1','l2']
            C = [10,100,1000]
            hyperparameters = dict(C=C,penalty=penalty)
            
            clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
            theta_hat = clf.fit(train_data,train_labels)
            
            mean_e_out = 1 - theta_hat.score(val_data_fold,val_label_fold)

            num_1 = np.round_(mean_e_out*len(val_data_fold),2)
            num_0 = len(val_data_fold)-num_1
            fake_sample = []
            for i in range(int(num_1)):
                fake_sample.append(1)
            for i in range(int(num_0)):
                fake_sample.append(0)
            var_e_out = statistics.variance(fake_sample)
            a_list.append((np.mean(e_in) - mean_e_out)**2)
            b_list.append(statistics.variance(fake_sample)/len(val_data_fold))
            es.append(e_in)
    

    MSEhat = np.mean(a_list) - np.mean(b_list)
    errhat_NCV = np.mean(es)
    return errhat_NCV, MSEhat

In [ ]:
import scipy.stats as st
import random

#CREATING PLOTS WITH NCV

# LINEAR CLASSIFIER

#specifically works for flashes
def classifier_accuracy_by_trial_window_ncv_lin(flashes_presentation_table, lfp_data, window_size, end_time, time_res, reps, k, alpha):
    classifier_accuracies_qalphadiv2 = []
    classifier_accuracies_means = []
    classifier_accuracies_q1minusalphadiv2 = []
    errhat_ncv_store = []
    errhat_cv_store = []
        
    
    window_start_times = []
    start_time = 0
    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    print(window_start_times)
    
    model_coefficients = np.zeros((len(lfp_data.channel),int(end_time/time_res)))
    print("model coefficients shape:",model_coefficients.shape)

    next_empty_column = 0
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
    
 


    #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
            b = aligned_lfp_1,
            axis = 1,
            equal_var = False,
            nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)



        errhat_ncv, MSEhat = nested_CV(X=flashes_data_combined, Y=labels, R=reps, K=k)
        
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
        


        errhat_cv = np.mean(standard_CV(flashes_data_combined,labels,k))
        
        
        errhat_ncv_store.append(errhat_ncv)
        errhat_cv_store.append(errhat_cv)
        
        biashat = (1+((k-2)/k))*(errhat_ncv-errhat_cv)
        point_estimate = errhat_ncv - biashat
        
        
        #CONSTRUCTION
        print("window:",window_start)
        print("k:",k,"MSEhat",MSEhat)
        print("NCV error estimate:",errhat_ncv,"point estimate:",point_estimate)
        
        #/CONSTRUCTION
        
        
        interval_radius = st.norm.ppf(1-alpha/2)*math.sqrt((k-1)/k)*math.sqrt(abs(MSEhat))     
        
    
#         coefficient_store = np.matrix(coefficient_store)
#         print(coefficient_store.shape)
#         coefficient_store = np.transpose(coefficient_store)
#         print(coefficient_store.shape)
#         coefficient_store = coefficient_store.mean(axis=1)
#         print(coefficient_store.shape)


        
    

        classifier_accuracies_qalphadiv2.append(min(1,1-(point_estimate - interval_radius)))
        classifier_accuracies_means.append(1-point_estimate)
        classifier_accuracies_q1minusalphadiv2.append(min(1,1-(point_estimate + interval_radius)))
        
    return window_start_times, classifier_accuracies_qalphadiv2, classifier_accuracies_means, classifier_accuracies_q1minusalphadiv2, errhat_ncv_store, errhat_cv_store

#-----------------------------------------------------------------------------------------

starts, ci_low, classifier_accuracies, ci_hi, errhat_ncv, errhat_cv = classifier_accuracy_by_trial_window_ncv_lin(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         reps=10, k=5,
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Logistic regression classifer accuracy with 95% CI from NCV')
_ = plt.legend()

# plt.figure(figsize=(8,6))
# im = plt.plot(starts, errhat_ncv, label="mean")
# _ = plt.xlabel('Start of time window following stimulus onset (s)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
# _ = plt.legend()

# plt.figure(figsize=(8,6))
# im = plt.plot(starts, errhat_cv, label="mean")
# _ = plt.xlabel('Start of time window following stimulus onset (s)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
# _ = plt.legend()

# fig = plt.figure(figsize=(8,6))
# im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
# _ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
# _ = plt.ylabel("Electrode index")
# _ = plt.title("Linear Classifier Coefficients")
# _ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

In [ ]:
st.norm.ppf(.975)
statistics.variance([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0])


In [ ]:
# COMBINED THALAMUS AND V1
# LINEAR CLASSIFIER

#specifically works for flashes
def classifier_accuracy_by_trial_window_boot_lin_v1_th(flashes_presentation_table, lfp_data_v1, lfp_data_th, window_size, end_time, time_res, num_bootstrap_iterations, alpha):
    classifier_accuracies_qalphadiv2 = []
    classifier_accuracies_means = []
    classifier_accuracies_q1minusalphadiv2 = []
    
    window_start_times = []
    start_time = 0
    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    print(window_start_times)
    
    model_coefficients = np.zeros((len(lfp_data_v1.channel),int(end_time/time_res)))
    print("model coefficients shape:",model_coefficients.shape)

    next_empty_column = 0
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
    
 
    #V1

    #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data_v1.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1_v1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data_v1.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1_v1 = ds['aligned_lfp']
        
        
        #thalamus
        
            #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data_th.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1_th = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data_th.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1_th = ds['aligned_lfp']
        
        


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

#         tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
#             b = aligned_lfp_1,
#             axis = 1,
#             equal_var = False,
#             nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data_v1 = aligned_lfp_1_v1.to_masked_array()
        flashes_minus1_data_v1 = aligned_lfp_minus1_v1.to_masked_array()
        flashes_1_data_v1.mask = np.ma.nomask
        flashes_minus1_data_v1.mask = np.ma.nomask
        flashes_data_combined_v1 = np.concatenate((flashes_1_data_v1,flashes_minus1_data_v1),axis=1).transpose(1,0,2)
        flashes_data_combined_v1 = flashes_data_combined_v1.reshape(flashes_data_combined_v1.shape[0],flashes_data_combined_v1.shape[1] * flashes_data_combined_v1.shape[2])
        
        flashes_1_data_th = aligned_lfp_1_th.to_masked_array()
        flashes_minus1_data_th = aligned_lfp_minus1_th.to_masked_array()
        flashes_1_data_th.mask = np.ma.nomask
        flashes_minus1_data_th.mask = np.ma.nomask
        flashes_data_combined_th = np.concatenate((flashes_1_data_th,flashes_minus1_data_th),axis=1).transpose(1,0,2)
        flashes_data_combined_th = flashes_data_combined_th.reshape(flashes_data_combined_th.shape[0],flashes_data_combined_th.shape[1] * flashes_data_combined_th.shape[2])

        flashes_data_combined = np.concatenate((flashes_data_combined_v1,flashes_data_combined_th),axis=1)
        
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)


    # PERFORM MANY CLASSIFICATIONS ON BOOTSTRAPPED SAMPLES

        window_classifier_accuracies = []
        coefficient_store = []
        
        for i in range(num_bootstrap_iterations):
                       
            bootstrapped_sample = []
            bootstrapped_labels = []
            
            while len(bootstrapped_sample) < len(flashes_data_combined):
                rand_ind = np.random.choice(range(len(flashes_data_combined)))
                bootstrapped_sample.append(flashes_data_combined[rand_ind])
                bootstrapped_labels.append(labels[rand_ind])


            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)

        # TUNE HYPERPARAMETERS USING CROSS-VALIDATION

            model = sklearn.linear_model.LogisticRegression(solver='liblinear')

            penalty = ['l1','l2']
            C = [10,100,1000]
            hyperparameters = dict(C=C,penalty=penalty)
            
            clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
            best_model = clf.fit(x_train,y_train)
            
            
        # EXTRACT MODEL COEFFICIENTS FOR VISUALIZATION  
            
#             coefficient_store.append(best_model.best_estimator_.coef_[0])



# TESTING

            window_classifier_accuracies.append(best_model.score(x_test,y_test))
    
#         coefficient_store = np.matrix(coefficient_store)
#         print(coefficient_store.shape)
#         coefficient_store = np.transpose(coefficient_store)
#         print(coefficient_store.shape)
#         coefficient_store = coefficient_store.mean(axis=1)
#         print(coefficient_store.shape)

        for i in range(len(coefficient_store)):
            print(i)
#             model_coefficients[i,next_empty_column] = coefficient_store[i]
        next_empty_column += 1
        
    
        window_classifier_accuracies = sorted(window_classifier_accuracies)
        classifier_accuracies_qalphadiv2.append(window_classifier_accuracies[math.floor(len(window_classifier_accuracies)*alpha/2)])
        classifier_accuracies_means.append(np.mean(window_classifier_accuracies))
        classifier_accuracies_q1minusalphadiv2.append(window_classifier_accuracies[math.ceil((1-alpha/2)*len(window_classifier_accuracies))])
        
    return window_start_times, classifier_accuracies_qalphadiv2, classifier_accuracies_means, classifier_accuracies_q1minusalphadiv2, model_coefficients

#-----------------------------------------------------------------------------------------

starts, ci_low, classifier_accuracies, ci_hi, coefficients = classifier_accuracy_by_trial_window_boot_lin_v1_th(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data_v1=lfp_VISp, lfp_data_th=lfp_thalamus,
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=100, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
_ = plt.legend()

fig = plt.figure(figsize=(8,6))
im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
_ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
_ = plt.ylabel("Electrode index")
_ = plt.title("Linear Classifier Coefficients")
_ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

In [ ]:
# RF
import sklearn.ensemble
#specifically works for flashes
def classifier_accuracy_by_trial_window_boot_rf(flashes_presentation_table, lfp_data, window_size, end_time, time_res, num_bootstrap_iterations, alpha):
    classifier_accuracies_qalphadiv2 = []
    classifier_accuracies_means = []
    classifier_accuracies_q1minusalphadiv2 = []
    
    window_start_times = []
    start_time = 0
    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
        print(window_start)
    
 


    #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
            b = aligned_lfp_1,
            axis = 1,
            equal_var = False,
            nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)


    # PERFORM MANY CLASSIFICATIONS ON BOOTSTRAPPED SAMPLES

        window_classifier_accuracies = []
        
        for i in range(num_bootstrap_iterations):
                       
            bootstrapped_sample = []
            bootstrapped_labels = []
            
            while len(bootstrapped_sample) < len(flashes_data_combined):
                rand_ind = np.random.choice(range(len(flashes_data_combined)))
                bootstrapped_sample.append(flashes_data_combined[rand_ind])
                bootstrapped_labels.append(labels[rand_ind])


            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)

        # TUNE HYPERPARAMETERS USING CROSS-VALIDATION

            model = sklearn.ensemble.RandomForestClassifier(bootstrap = False)

            criterion = ['gini','entropy']
           
            hyperparameters = dict(criterion = criterion)

            clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
            best_model = clf.fit(x_train,y_train)



# TESTING

            window_classifier_accuracies.append(best_model.score(x_test,y_test))
        window_classifier_accuracies = sorted(window_classifier_accuracies)
        classifier_accuracies_qalphadiv2.append(window_classifier_accuracies[math.floor(len(window_classifier_accuracies)*alpha/2)])
        classifier_accuracies_means.append(np.mean(window_classifier_accuracies))
        classifier_accuracies_q1minusalphadiv2.append(window_classifier_accuracies[math.ceil((1-alpha/2)*len(window_classifier_accuracies))])
        
    return window_start_times, classifier_accuracies_qalphadiv2, classifier_accuracies_means, classifier_accuracies_q1minusalphadiv2

#-----------------------------------------------------------------------------------------

starts, ci_low, classifier_accuracies, ci_hi = classifier_accuracy_by_trial_window_boot_rf(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=100, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Window End (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.legend()

In [ ]:
# SVC

#specifically works for flashes
def classifier_accuracy_by_trial_window_boot_svc(flashes_presentation_table, lfp_data, window_size, end_time, time_res, num_bootstrap_iterations, alpha):
    classifier_accuracies_qalphadiv2 = []
    classifier_accuracies_means = []
    classifier_accuracies_q1minusalphadiv2 = []
    
    window_start_times = []
    start_time = 0

    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
    
 


#for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
            b = aligned_lfp_1,
            axis = 1,
            equal_var = False,
            nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)


    # PERFORM MANY CLASSIFICATIONS ON BOOTSTRAPPED SAMPLES

        window_classifier_accuracies = []
        
        for i in range(num_bootstrap_iterations):
                       
            bootstrapped_sample = []
            bootstrapped_labels = []
            
            while len(bootstrapped_sample) < len(flashes_data_combined):
                rand_ind = np.random.choice(range(len(flashes_data_combined)))
                bootstrapped_sample.append(flashes_data_combined[rand_ind])
                bootstrapped_labels.append(labels[rand_ind])


            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)

        # TUNE HYPERPARAMETERS USING CROSS-VALIDATION

            model = sklearn.svm.SVC()

            kernel = ['linear', 'poly', 'rbf', 'sigmoid']
            C = [10,100,1000]
            hyperparameters = dict(C=C,kernel=kernel)

            clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
            print(clf.best_params_)
            best_model = clf.fit(x_train,y_train)



# TESTING

            window_classifier_accuracies.append(best_model.score(x_test,y_test))
        window_classifier_accuracies = sorted(window_classifier_accuracies)
        classifier_accuracies_qalphadiv2.append(window_classifier_accuracies[math.floor(len(window_classifier_accuracies)*alpha/2)])
        classifier_accuracies_means.append(np.mean(window_classifier_accuracies))
        classifier_accuracies_q1minusalphadiv2.append(window_classifier_accuracies[math.ceil((1-alpha/2)*len(window_classifier_accuracies))])
        
    return window_start_times, classifier_accuracies_qalphadiv2, classifier_accuracies_means, classifier_accuracies_q1minusalphadiv2

#-----------------------------------------------------------------------------------------

starts, ci_low, classifier_accuracies, ci_hi = classifier_accuracy_by_trial_window_boot_svc(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=100, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Window End (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.legend()

In [ ]:
print(classifier_accuracies)

In [ ]:
print(coefficients)

In [ ]:
# LINEAR CLASSIFIER

#specifically works for flashes
def classifier_accuracy_by_trial_window_boot_lin(flashes_presentation_table, lfp_data, window_size, end_time, time_res, num_bootstrap_iterations, alpha):
    classifier_accuracies_qalphadiv2 = []
    classifier_accuracies_means = []
    classifier_accuracies_q1minusalphadiv2 = []
    
    window_start_times = []
    start_time = 0
    while start_time + 1000*window_size < 1000*end_time:
        window_start_times.append(start_time/1000)
        start_time += 1000*window_size
    print(window_start_times)
    
    model_coefficients = np.zeros((len(lfp_data.channel),int(end_time/time_res)))
    print("model coefficients shape:",model_coefficients.shape)

    next_empty_column = 0
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA

    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color
                                                                      == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color
                                                                          == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values

    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values

    for window_start in window_start_times:
    
 


    #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

#for -1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        if 1/(len(trial_window)*100) <  time_res:
            trial_window = trial_window[:-1]
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window),
                                      names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']


# COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS

        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
            b = aligned_lfp_1,
            axis = 1,
            equal_var = False,
            nan_policy = 'raise')

# FORMAT TRIAL DATA FOR CLASSIFICATION


        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)


    # PERFORM MANY CLASSIFICATIONS ON BOOTSTRAPPED SAMPLES

        window_classifier_accuracies = []
        coefficient_store = []
        
        for i in range(num_bootstrap_iterations):
                       
            bootstrapped_sample = []
            bootstrapped_labels = []
            
            while len(bootstrapped_sample) < len(flashes_data_combined):
                rand_ind = np.random.choice(range(len(flashes_data_combined)))
                bootstrapped_sample.append(flashes_data_combined[rand_ind])
                bootstrapped_labels.append(labels[rand_ind])


            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)

        # TUNE HYPERPARAMETERS USING CROSS-VALIDATION

            model = sklearn.linear_model.LogisticRegression(solver='liblinear', penalty='l2')

#             penalty = ['l1','l2']
            C = [10,100,1000]
            hyperparameters = dict(C=C)
            
            clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
            best_model = clf.fit(x_train,y_train)
            
            
        # EXTRACT MODEL COEFFICIENTS FOR VISUALIZATION  
            
            coefficient_store.append(best_model.best_estimator_.coef_[0])



# TESTING

            window_classifier_accuracies.append(best_model.score(x_test,y_test))
    
        coefficient_store = np.matrix(coefficient_store)
        print(coefficient_store.shape)
        coefficient_store = np.transpose(coefficient_store)
        print(coefficient_store.shape)
        coefficient_store = coefficient_store.mean(axis=1)
        print(coefficient_store.shape)

        for i in range(len(coefficient_store)):
            print(i)
            model_coefficients[i,next_empty_column] = coefficient_store[i]
        next_empty_column += 1
        
    
        window_classifier_accuracies = sorted(window_classifier_accuracies)
        classifier_accuracies_qalphadiv2.append(window_classifier_accuracies[math.floor(len(window_classifier_accuracies)*alpha/2)])
        classifier_accuracies_means.append(np.mean(window_classifier_accuracies))
        classifier_accuracies_q1minusalphadiv2.append(window_classifier_accuracies[math.ceil((1-alpha/2)*len(window_classifier_accuracies))])
        
    return window_start_times, classifier_accuracies_qalphadiv2, classifier_accuracies_means, classifier_accuracies_q1minusalphadiv2, model_coefficients

#-----------------------------------------------------------------------------------------

starts, ci_low, classifier_accuracies, ci_hi, coefficients = classifier_accuracy_by_trial_window_boot_lin(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=100, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI, trained on 10 ms sliding time windows')
_ = plt.legend()

fig = plt.figure(figsize=(8,6))
im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
_ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
_ = plt.ylabel("Electrode index")
_ = plt.title("Linear Classifier Coefficients")
_ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

In [ ]:
#specifically works for flashes
def classifier_accuracy_by_trial_window_boot_mod(flashes_presentation_table, lfp_data, window_size, end_time, time_res, num_bootstrap_iterations, alpha):
    classifier_accuracies_qalphadiv2 = []
    classifier_accuracies_means = []
    classifier_accuracies_q1minusalphadiv2 = []
    
    window_start_times = []
    start_time = 0
    while start_time + window_size < end_time:
        window_start_times.append(start_time)
        start_time += window_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA
    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                          == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                              == -1.0]
    presentation_times_1 = flashes_presentation_table_1.start_time.values
    presentation_ids_1 = flashes_presentation_table_1.index.values
    
    presentation_times_minus1 = flashes_presentation_table_minus1.start_time.values
    presentation_ids_minus1 = flashes_presentation_table_minus1.index.values
    
    for window_start in window_start_times:
    
     


        #for 1



        trial_window = np.arange(window_start, window_start + window_size, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times_1])

        inds = pd.MultiIndex.from_product((presentation_ids_1, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

        #for -1

       

        trial_window = np.arange(window_start, window_start + window_size, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

        inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']
        
        
        # COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS
        
        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
               b = aligned_lfp_1,
               axis = 1,
               equal_var = False,
               nan_policy = 'raise')
        
        # FORMAT TRIAL DATA FOR CLASSIFICATION
        
        
        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)
            

        # PERFORM MANY CLASSIFICATIONS ON BOOTSTRAPPED SAMPLES
        
        window_classifier_accuracies = []
               
            
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)

        # TUNE HYPERPARAMETERS USING CROSS-VALIDATION
        
        model = sklearn.linear_model.LogisticRegression(solver='saga')

        penalty = ['l1','l2']
        C = np.logspace(10,1000)
        hyperparameters = dict(C=C,penalty=penalty)
        
        clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=5)
        best_model = clf.fit(x_train,y_train)
        
        print(clf)

        # TESTING

        window_classifier_accuracies.append(best_model.score(x_test,y_test))
        window_classifier_accuracies = sorted(window_classifier_accuracies)
#         classifier_accuracies_qalphadiv2.append(window_classifier_accuracies[math.floor(len(window_classifier_accuracies)*alpha/2)])
        classifier_accuracies_means.append(np.mean(window_classifier_accuracies))
#         classifier_accuracies_q1minusalphadiv2.append(window_classifier_accuracies[math.ceil((1-alpha/2)*len(window_classifier_accuracies))])
        
    return window_start_times, classifier_accuracies_means

#         classifier_accuracies_means.append(window_classifier_accuracies)
    
#     return classifier_accuracies_means


# v----------------------------------------------------------------------------------------------------

starts, classifier_accuracies = classifier_accuracy_by_trial_window_boot_mod(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=100, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*widths[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")

_ = plt.xlabel('Window End (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.legend()

In [ ]:
# MEGACELL FOR INCREASING WINDOWS FOR VISUAL AREAS ALL 3 CLASSIFIERS

#RF
def classifier_accuracy_by_time_after_presentation_rf(flashes_presentation_table, lfp_data, step_size, end_time, time_res, num_classification_iterations):
    mean_classifier_accuracies = []
    
    window_end_times = []
    end_time_temp = step_size
    while end_time_temp <= end_time:
        window_end_times.append(end_time_temp)
        end_time_temp += step_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA
    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                          == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                              == -1.0]
    
    
    for window_end in window_end_times:
    
     


        #for 1

        presentation_times = flashes_presentation_table_1.start_time.values
        presentation_ids = flashes_presentation_table_1.index.values

        trial_window = np.arange(0, window_end, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

        #for -1

        presentation_times = flashes_presentation_table_minus1.start_time.values
        presentation_ids = flashes_presentation_table_minus1.index.values

        trial_window = np.arange(0, window_end, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']
        
        
        # COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS
        
        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
               b = aligned_lfp_1,
               axis = 1,
               equal_var = False,
               nan_policy = 'raise')
        
        # FORMAT TRIAL DATA FOR CLASSIFICATION
        
        
        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)

        # PERFORM MANY CLASSIFICATIONS
        
        window_classifier_accuracies = []
        for i in range(num_classification_iterations):
            
            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
        
            model = sklearn.ensemble.RandomForestClassifier(bootstrap = False, criterion='entropy')

            model.fit(x_train,y_train)



# TESTING

            window_classifier_accuracies.append(model.score(x_test,y_test))
        mean_classifier_accuracies.append(np.mean(window_classifier_accuracies))
        
    return window_end_times, mean_classifier_accuracies


#SVC
def classifier_accuracy_by_time_after_presentation_svc(flashes_presentation_table, lfp_data, step_size, end_time, time_res, num_classification_iterations):
    mean_classifier_accuracies = []
    
    window_end_times = []
    end_time_temp = step_size
    while end_time_temp <= end_time:
        window_end_times.append(end_time_temp)
        end_time_temp += step_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA
   ncv
#LIN

def classifier_accuracy_by_time_after_presentation_lin(flashes_presentation_table, lfp_data, step_size, end_time, time_res, num_classification_iterations):
    mean_classifier_accuracies = []
    
    window_end_times = []
    end_time_temp = step_size
    while end_time_temp <= end_time:
        window_end_times.append(end_time_temp)
        end_time_temp += step_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA
    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                          == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                              == -1.0]
    
    
    for window_end in window_end_times:
    
     


        #for 1

        presentation_times = flashes_presentation_table_1.start_time.values
        presentation_ids = flashes_presentation_table_1.index.values

        trial_window = np.arange(0, window_end, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

        #for -1

        presentation_times = flashes_presentation_table_minus1.start_time.values
        presentation_ids = flashes_presentation_table_minus1.index.values

        trial_window = np.arange(0, window_end, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']
        
        
        # COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS
        
        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
               b = aligned_lfp_1,
               axis = 1,
               equal_var = False,
               nan_policy = 'raise')
        
        # FORMAT TRIAL DATA FOR CLASSIFICATION
        
        
        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)

        # PERFORM MANY CLASSIFICATIONS
        
        window_classifier_accuracies = []
        for i in range(num_classification_iterations):
            
            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
        
            # TUNE HYPERPARAMETERS USING CROSS-VALIDATION

            model = sklearn.linear_model.LogisticRegression(solver='liblinear')

            penalty = ['l1','l2']
            C = [10,100,1000]
            hyperparameters = dict(C=C,penalty=penalty)
            
            clf = sklearn.model_selection.GridSearchCV(model, hyperparameters, cv=3)
            best_model = clf.fit(x_train,y_train)



# TESTING

            window_classifier_accuracies.append(best_model.score(x_test,y_test))
        mean_classifier_accuracies.append(np.mean(window_classifier_accuracies))
        
    return window_end_times, mean_classifier_accuracies


#-------------------------------P L O T T I N G-----------------------------------------------

  #     RF
ends_pm, means_pm = classifier_accuracy_by_time_after_presentation_rf(flashes_presentation_table=flashes_presentation_table, 
                                                  lfp_data=lfp_VISpm, 
                                                  step_size=0.01, 
                                                  end_time=0.5, 
                                                  time_res=1/100, 
                                                  num_classification_iterations=50)
ends_l, means_l = classifier_accuracy_by_time_after_presentation_rf(flashes_presentation_table=flashes_presentation_table, 
                                                  lfp_data=lfp_VISl, 
                                                  step_size=0.01, 
                                                  end_time=0.5, 
                                                  time_res=1/100, 
                                                  num_classification_iterations=50)
ends_p, means_p = classifier_accuracy_by_time_after_presentation_rf(flashes_presentation_table=flashes_presentation_table, 
                                                  lfp_data=lfp_VISp, 
                                                  step_size=0.01, 
                                                  end_time=0.5, 
                                                  time_res=1/100, 
                                                  num_classification_iterations=50)
ends_rl, means_rl = classifier_accuracy_by_time_after_presentation_rf(flashes_presentation_table=flashes_presentation_table, 
                                                  lfp_data=lfp_VISrl, 
                                                  step_size=0.01, 
                                                  end_time=0.5, 
                                                  time_res=1/100, 
                                                  num_classification_iterations=50)

plt.figure(figsize=(8,6))
im = plt.plot(ends_pm, means_pm, label="PM")
im = plt.plot(ends_l, means_l, label="LM")
im = plt.plot(ends_p, means_p, label="V1")
im = plt.plot(ends_rl, means_rl, label="RL")
_ = plt.xlabel('Window Width (by 10ms)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.legend()
_ = plt.title('Random Forest Classifier Accuracy for Visual Areas by Time Following Stimulus')

#        SVC

# ends_pm, means_pm = classifier_accuracy_by_time_after_presentation_svc(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISpm, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)
# ends_l, means_l = classifier_accuracy_by_time_after_presentation_svc(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISl, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)
# ends_p, means_p = classifier_accuracy_by_time_after_presentation_svc(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISp, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)
# ends_rl, means_rl = classifier_accuracy_by_time_after_presentation_svc(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISrl, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)

# plt.figure(figsize=(8,6))
# im = plt.plot(ends_pm, means_pm, label="PM")
# im = plt.plot(ends_l, means_l, label="LM")
# im = plt.plot(ends_p, means_p, label="V1")
# im = plt.plot(ends_rl, means_rl, label="RL")
# _ = plt.xlabel('Window Width (by 10ms)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.legend()
# _ = plt.title('Support Vector Classifier Accuracy for Visual Areas by Time Following Stimulus')


#        LIN

# ends_pm, means_pm = classifier_accuracy_by_time_after_presentation_lin(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISpm, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)
# ends_l, means_l = classifier_accuracy_by_time_after_presentation_lin(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISl, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)
# ends_p, means_p = classifier_accuracy_by_time_after_presentation_lin(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISp, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)
# ends_rl, means_rl = classifier_accuracy_by_time_after_presentation_lin(flashes_presentation_table=flashes_presentation_table, 
#                                                   lfp_data=lfp_VISrl, 
#                                                   step_size=0.01, 
#                                                   end_time=0.5, 
#                                                   time_res=1/100, 
#                                                   num_classification_iterations=50)

# plt.figure(figsize=(8,6))
# im = plt.plot(ends_pm, means_pm, label="PM")
# im = plt.plot(ends_l, means_l, label="LM")
# im = plt.plot(ends_p, means_p, label="V1")
# im = plt.plot(ends_rl, means_rl, label="RL")
# _ = plt.xlabel('Window Width (by 10ms)')
# _ = plt.ylabel('Classifier Accuracy')
# _ = plt.legend()
# _ = plt.title('Linear Classifier Accuracy for Visual Areas by Time Following Stimulus')

In [ ]:
print(len(means_l))

In [ ]:
out = classifier_accuracy_by_trial_window_boot(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=100, 
                                                                         alpha=.05)

    
# plt.hist(out[0])
plt.hist(out[42],20)


In [ ]:
starts, ci_low, classifier_accuracies, ci_hi = classifier_accuracy_by_trial_window_boot(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.05, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=100, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
im = plt.plot(starts, ci_low, label="ci low")
im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Window End (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.legend()

In [ ]:
starts, ci_low, classifier_accuracies, ci_hi = classifier_accuracy_by_trial_window_boot(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISl, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=500, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*widths[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
im = plt.plot(starts, ci_low, label="ci low")
im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Window End (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.legend()

In [ ]:
starts, ci_low, classifier_accuracies, ci_hi = classifier_accuracy_by_trial_window_boot(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISpm, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=250, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*widths[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
im = plt.plot(starts, ci_low, label="ci low")
im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Window End (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.legend()

# Evaluating classifier accuracy based on window of increasing size

In [ ]:
#specifically works for flashes
def classifier_accuracy_by_time_after_presentation(flashes_presentation_table, lfp_data, step_size, end_time, time_res, num_classification_iterations):
    mean_classifier_accuracies = []
    
    window_end_times = []
    end_time_temp = step_size
    while end_time_temp <= end_time:
        window_end_times.append(end_time_temp)
        end_time_temp += step_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA
    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                          == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                              == -1.0]
    
    
    for window_end in window_end_times:
    
     


        #for 1

        presentation_times = flashes_presentation_table_1.start_time.values
        presentation_ids = flashes_presentation_table_1.index.values

        trial_window = np.arange(0, window_end, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

        #for -1

        presentation_times = flashes_presentation_table_minus1.start_time.values
        presentation_ids = flashes_presentation_table_minus1.index.values

        trial_window = np.arange(0, window_end, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']
        
        
        # COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS
        
        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
               b = aligned_lfp_1,
               axis = 1,
               equal_var = False,
               nan_policy = 'raise')
        
        # FORMAT TRIAL DATA FOR CLASSIFICATION
        
        
        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)

        # PERFORM MANY CLASSIFICATIONS
        
        window_classifier_accuracies = []
        for i in range(num_classification_iterations):
            
            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
        
            model = sklearn.linear_model.LogisticRegression()
            model.fit(x_train,y_train)
            window_classifier_accuracies.append(model.score(x_test,y_test))
        mean_classifier_accuracies.append(np.mean(window_classifier_accuracies))
        
    return window_end_times, mean_classifier_accuracies

In [ ]:
widths, classifier_accuracies = classifier_accuracy_by_time_after_presentation(flashes_presentation_table=flashes_presentation_table, 
                                                                         lfp_data=lfp_VISp, 
                                                                         step_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/1000, 
                                                                         num_classification_iterations=100)
for i in range(len(widths)):
    print("Window width: {}       Mean Classifier Accuracy: {}".format(widths[i], classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(widths, classifier_accuracies)
_ = plt.xlabel('Window length (ms)')
_ = plt.ylabel('Classifier Accuracy')

# EVALUATING CLASSIFIER ACCURACY BASED ON SLIDING TIME WINDOW

In [ ]:
#specifically works for flashes
def classifier_accuracy_by_trial_window(flashes_presentation_table, lfp_data, window_size, end_time, time_res, num_classification_iterations):
    mean_classifier_accuracies = []
    quant_low = []
    quant_high = []
    
    window_start_times = []
    start_time = 0
    while start_time + window_size < end_time:
        window_start_times.append(start_time)
        start_time += window_size
    
    
# PRODUCE ALIGNED LFPs FROM PROBE DATA
    flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                          == 1.0]
    flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                              == -1.0]
    
    
    for window_start in window_start_times:
    
     


        #for 1

        presentation_times = flashes_presentation_table_1.start_time.values
        presentation_ids = flashes_presentation_table_1.index.values

        trial_window = np.arange(window_start, window_start + window_size, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_1 = ds['aligned_lfp']

        #for -1

        presentation_times = flashes_presentation_table_minus1.start_time.values
        presentation_ids = flashes_presentation_table_minus1.index.values

        trial_window = np.arange(window_start, window_start + window_size, time_res)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp_minus1 = ds['aligned_lfp']
        
        
        # COMPUTE T SCORES AND CORRESPONDING P VALUES IN INDEPENDENT T TEST BETWEEN TRIAL CONDITIONS
        
        tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
               b = aligned_lfp_1,
               axis = 1,
               equal_var = False,
               nan_policy = 'raise')
        
        # FORMAT TRIAL DATA FOR CLASSIFICATION
        
        
        flashes_1_data = aligned_lfp_1.to_masked_array()
        flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
        flashes_1_data.mask = np.ma.nomask
        flashes_minus1_data.mask = np.ma.nomask
        flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
        flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
        labels = []
        for i in range(75):
            labels.append(1)
        for i in range(75,150):
            labels.append(0)

#         print(len(flashes_data_combined))
#         print(len(flashes_data_combined[0]))
#         print(len(labels))
        # PERFORM MANY CLASSIFICATIONS
        
        window_classifier_accuracies = []
        for i in range(num_classification_iterations):
            
            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
        
            model = sklearn.linear_model.LogisticRegression()
            model.fit(x_train,y_train)
            window_classifier_accuracies.append(model.score(x_test,y_test))
#         mean_classifier_accuracies.append(np.mean(window_classifier_accuracies))
#         sort = sorted(window_classifier_accuracies)
#         quant_low.append(sort[math.floor(.025 * len(sort))])
#         quant_high.append(sort[math.ceil(.975 * len(sort))])
        
#     return window_start_times, quant_low, mean_classifier_accuracies, quant_high

        mean_classifier_accuracies.append(window_classifier_accuracies)
    
    return mean_classifier_accuracies

In [ ]:
out = classifier_accuracy_by_trial_window(flashes_presentation_table=flashes_presentation_table, 
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_classification_iterations=100)
plt.hist(out[41])


In [ ]:
start_times, classifier_accuracies = classifier_accuracy_by_trial_window(flashes_presentation_table=flashes_presentation_table, 
                                                                         lfp_data=lfp_VISpm, 
                                                                         window_size=.02, 
                                                                         end_time=.5, 
                                                                         time_res=1/1000, 
                                                                         num_classification_iterations=100)
for i in range(len(start_times)):
    print("Window start: {}       Mean Classifier Accuracy: {}".format(start_times[i], classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(start_times, classifier_accuracies)
_ = plt.xlabel('Start Time')
_ = plt.ylabel('Classifier Accuracy')

In [ ]:
start_times, classifier_accuracies = classifier_accuracy_by_trial_window(flashes_presentation_table=flashes_presentation_table, 
                                                                         lfp_data=lfp_VISam, 
                                                                         window_size=.02, 
                                                                         end_time=.5, 
                                                                         time_res=1/1000, 
                                                                         num_classification_iterations=100)
for i in range(len(start_times)):
    print("Window start: {}       Mean Classifier Accuracy: {}".format(start_times[i], classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(start_times, classifier_accuracies)
_ = plt.xlabel('Start Time')
_ = plt.ylabel('Classifier Accuracy')

In [ ]:
start_times, classifier_accuracies = classifier_accuracy_by_trial_window(flashes_presentation_table=flashes_presentation_table, 
                                                                         lfp_data=lfp_VISl, 
                                                                         window_size=.02, 
                                                                         end_time=.5, 
                                                                         time_res=1/1000, 
                                                                         num_classification_iterations=100)
for i in range(len(start_times)):
    print("Window start: {}       Mean Classifier Accuracy: {}".format(start_times[i], classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(start_times, classifier_accuracies)
_ = plt.xlabel('Start Time')
_ = plt.ylabel('Classifier Accuracy')

In [ ]:
start_times, classifier_accuracies = classifier_accuracy_by_trial_window(flashes_presentation_table=flashes_presentation_table, 
                                                                         lfp_data=lfp_VISrl, 
                                                                         window_size=.005, 
                                                                         end_time=.5, 
                                                                         time_res=1/1000, 
                                                                         num_classification_iterations=100)
for i in range(len(start_times)):
    print("Window start: {}       Mean Classifier Accuracy: {}".format(start_times[i], classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(start_times, classifier_accuracies)
_ = plt.xlabel('Start Time')
_ = plt.ylabel('Classifier Accuracy')

In [ ]:
starts, ci_low, classifier_accuracies, ci_hi, coefficients = classifier_accuracy_by_trial_window_boot_lin(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_VISp, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=250, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI for V1')
_ = plt.legend()

fig = plt.figure(figsize=(8,6))
im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
_ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
_ = plt.ylabel("Electrode index")
_ = plt.title("Linear Classifier Coefficients")
_ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

In [ ]:
starts, ci_low, classifier_accuracies, ci_hi, coefficients = classifier_accuracy_by_trial_window_boot_lin(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_LP, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=250, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI for LP')
_ = plt.legend()

fig = plt.figure(figsize=(8,6))
im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
_ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
_ = plt.ylabel("Electrode index")
_ = plt.title("Linear Classifier Coefficients")
_ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

In [ ]:
starts, ci_low, classifier_accuracies, ci_hi, coefficients = classifier_accuracy_by_trial_window_boot_lin(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_LGd, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=250, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI for LGd')
_ = plt.legend()

fig = plt.figure(figsize=(8,6))
im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
_ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
_ = plt.ylabel("Electrode index")
_ = plt.title("Linear Classifier Coefficients")
_ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

In [ ]:
starts, ci_low, classifier_accuracies, ci_hi, coefficients = classifier_accuracy_by_trial_window_boot_lin(flashes_presentation_table=flashes_presentation_table,
                                                                         lfp_data=lfp_thalamus, 
                                                                         window_size=.01, 
                                                                         end_time=.5, 
                                                                         time_res=1/100, 
                                                                         num_bootstrap_iterations=250, 
                                                                         alpha=.05)
for i in range(len(starts)):
    print("Window start: {} ms      Mean Classifier Accuracy: {}".format(math.floor(1000*starts[i]), classifier_accuracies[i]))
    
plt.figure(figsize=(8,6))
im = plt.plot(starts, classifier_accuracies, label="mean")
_ = plt.fill_between(starts, ci_low, ci_hi, alpha=0.2)
# im = plt.plot(starts, ci_low, label="ci low")
# im = plt.plot(starts, ci_hi, label="ci high")
_ = plt.xlabel('Start of time window following stimulus onset (s)')
_ = plt.ylabel('Classifier Accuracy')
_ = plt.title('Bootstrapped mean linear classifer accuracy with 95% CI for thalamus')
_ = plt.legend()

fig = plt.figure(figsize=(8,6))
im2 = plt.imshow(coefficients, origin='lower', cmap='bone')
_ = plt.xlabel("Time sample after stimulus onset (by 10ms)")
_ = plt.ylabel("Electrode index")
_ = plt.title("Linear Classifier Coefficients")
_ = plt.colorbar(im2, cax = fig.add_axes([.92, 0.33, 0.03, 0.38]))

# CLASSIFIER FOR FLASH COLOR

In [ ]:

flashes_presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'flashes']

flashes_presentation_times = flashes_presentation_table.start_time.values
flashes_presentation_ids = flashes_presentation_table.index.values


In [ ]:
lfp_VISp = lfp.sel(channel = lfp.channel[62:84])

flashes_presentation_table_1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                      == 1.0]
flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                      == -1.0]
#for overall
presentation_times = flashes_presentation_table.start_time.values
presentation_ids = flashes_presentation_table.index.values

trial_window = np.arange(0,.5, 1/100)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_VISp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

#for 1

presentation_times = flashes_presentation_table_1.start_time.values
presentation_ids = flashes_presentation_table_1.index.values

trial_window = np.arange(0,.5, 1/100)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_VISp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_1 = ds['aligned_lfp']

#for -1

presentation_times = flashes_presentation_table_minus1.start_time.values
presentation_ids = flashes_presentation_table_minus1.index.values

trial_window = np.arange(0,.5, 1/100)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_VISp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_minus1 = ds['aligned_lfp']

In [ ]:
aligned_lfp_minus1

In [ ]:
import scipy.stats as stats
import math

tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
               b = aligned_lfp_1,
               axis = 1,
               equal_var = False,
               nan_policy = 'raise')

# MANUAL BENJAMINI-HOCHBERG

def benjamini_hochberg(p_values, alpha):
    indices_of_significant_values = []
    p_values_flat = p_values.flatten()
    dictionary = dict(zip(p_values_flat,range(len(p_values_flat))))
    
    sorted_keys = sorted(dictionary)
    
    greatest_i_so_far = 0

    for i in range(len(sorted_keys)):
        key = sorted_keys[i]
        critical_value = i*alpha/len(dictionary)
        if key<critical_value:
            greatest_i_so_far = i
    for i in range(greatest_i_so_far):
        indices_of_significant_values.append(dictionary.get(sorted_keys[i]))

    indices_of_significant_values = sorted(indices_of_significant_values)
    print("number of significant p-values:",len(indices_of_significant_values))
    
    return indices_of_significant_values

significant_indices = benjamini_hochberg(p_values = pValue, alpha = .000000000000005)

asdf = []
for i in range(pValue.shape[0]*pValue.shape[1]):
    if i in significant_indices:
        asdf.append(1)
    else:
        asdf.append(0)
print(len(asdf))
mask = []
for i in range(pValue.shape[0]):
    row = []
    for j in range(pValue.shape[1]):
        row.append(asdf[i*pValue.shape[1]+j])
    mask.append(row)

fig = plt.figure(figsize=(8,6))
im = plt.imshow(mask, origin='lower')
_ = plt.xlabel('Time sample after stimulus onset (by 10ms)')
_ = plt.ylabel('Electrode index')
_ = plt.title('Benjamini-Hochberg-Corrected P-values for Independent T Test on LFP Data for Flash Colors')

import matplotlib
handles = [matplotlib.patches.Patch(facecolor='yellow', edgecolor='y',label='Significant feature')]
_ = plt.legend(handles=handles,loc='lower left')
# _ = plt.colorbar(im, cax = fig.add_axes([.92, 0.33, 0.03, 0.2]))
        

In [ ]:
# pValue.shape
print(aligned_lfp_1.presentation_id)
print(aligned_lfp_minus1.presentation_id)
print(aligned_lfp.presentation_id)
print(flashes_presentation_table.index.values)

In [ ]:
flashes_1_data = aligned_lfp_1.to_masked_array()
flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
flashes_1_data.mask = np.ma.nomask
flashes_minus1_data.mask = np.ma.nomask
flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
labels = []
for i in range(75):
    labels.append(1)
for i in range(75,150):
    labels.append(0)
print(flashes_data_combined.shape)
# print(labels)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
# print(x_train.shape)
# print(x_test.shape)
# print(len(y_train))
# print(len(y_test))
print(train_test_split)

In [ ]:
import sklearn.linear_model
model = sklearn.linear_model.LogisticRegression()
model.fit(x_train,y_train)
model.score(x_test,y_test)

# FIND TRIALS THAT ARE CONSISTENTLY MISCLASSIFIED
## NOTE: RUN CELL 2 ABOVE THIS ONE FIRST

In [ ]:
import sklearn.linear_model
# for entry in id_to_incorrect_count_dict:
#     print("Presentation ID: {}            Number incorrect: {}".format(str(entry),str(id_to_incorrect_count_dict[entry])))

In [ ]:
#MAKE DICTIONARY TO MAP LFP DATA TO PRESENTATION ID
data_to_id_dict = {}
flashes_data = aligned_lfp.to_masked_array()
flashes_data.mask = np.ma.nomask
flashes_data = flashes_data.transpose(1,0,2)
flashes_data = flashes_data.reshape(flashes_data.shape[0],flashes_data.shape[1]*flashes_data.shape[2])
#shape is now (presentation id, total features)

for id_ind in range(len(flashes_presentation_table.index.values)):
    id = flashes_presentation_table.index.values[id_ind]
    trial_lfp_data = flashes_data[id_ind,:]
    data_to_id_dict[trial_lfp_data.tobytes()] = id

#MAKE DICTIONARY TO MAP PRESENTATION ID TO NUMBER OF INCORRECTS
id_to_incorrect_count_dict = {}
for presentation_id in flashes_presentation_table.index.values:
    id_to_incorrect_count_dict[str(presentation_id)] = 0

num_iterations = 10000
print("Number of iterations: {}".format(str(num_iterations)))
    
for i in range(num_iterations):
    #DO RANDOM TRAIN-TEST SPLIT
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)

    #INITIALIZE AND TRAIN MODEL
    model = sklearn.linear_model.LogisticRegression()
    model.fit(x_train,y_train)
    
    #TEST EACH STIMULUS IN TEST SET OF MODEL
    predictions = model.predict(x_test)
    for prediction_ind in range(len(predictions)):
        prediction = predictions[prediction_ind]
        correct_output = y_test[prediction_ind]
        if prediction != correct_output:
            #increase trial ID's incorrect count by 1
            presentation_id = data_to_id_dict[x_test[prediction_ind].tobytes()]
            id_to_incorrect_count_dict[str(presentation_id)] += 1
            
for entry in id_to_incorrect_count_dict:
    print("Presentation ID: {}            Number incorrect: {}".format(str(entry),str(id_to_incorrect_count_dict[entry])))
    

# CLASSIFIER FOR DRIFTING GRATING ORIENTATION (JUST BETWEEN 315 AND 135)

In [ ]:

dg_presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'drifting_gratings']

dg_presentation_times = dg_presentation_table.start_time.values
dg_presentation_ids = dg_presentation_table.index.values


In [ ]:
import xarray as xr
lfp_VISp = lfp.sel(channel = lfp.channel[62:84])

dg_presentation_table_135 = dg_presentation_table[dg_presentation_table.orientation 
                                                                      == 135.0]
dg_presentation_table_315 = dg_presentation_table[dg_presentation_table.orientation 
                                                                      == 315.0]

presentation_times = dg_presentation_table_135.start_time.values
presentation_ids = dg_presentation_table_135.index.values

print(len(presentation_times))

trial_window = np.arange(0, 0.50, 1/100)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_VISp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_135 = ds['aligned_lfp']

presentation_times = dg_presentation_table_315.start_time.values
presentation_ids = dg_presentation_table_315.index.values

print(len(presentation_times[:-1]))

trial_window = np.arange(0, 0.50, 1/100)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_VISp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_315 = ds['aligned_lfp']

# aligned_lfp_135 = aligned_lfp_135[:,:8,:].append(aligned_lfp_135[:,12:14,:].append(aligned_lfp_135[:,19:,:]))
# aligned_lfp_315 = aligned_lfp_315[:,:8,:].append(aligned_lfp_315[:,12:14,:].append(aligned_lfp_315[:,19:,:]))

# aligned_lfp_135 = xr.concat(xr.concat(aligned_lfp_135[:,:8,:], 
#                                       aligned_lfp_135[:,12:14,:], dim='presentation_id'),
#                             aligned_lfp_135[:,19:,:], dim='presentation_id')
# aligned_lfp_315 = xr.concat(xr.concat(aligned_lfp_315[:,:8,:], 
#                                       aligned_lfp_315[:,12:14,:], dim='presentation_id'),
#                             aligned_lfp_315[:,19:,:], dim='presentation_id')

# xr.concat(aligned_lfp_135[:,:8,:], aligned_lfp_135[:,12:14,:], dim='presentation_id')

aligned_lfp_135 = aligned_lfp_135[:,:8,:].concat(aligned_lfp_135[:,12:14,:].concat(aligned_lfp_135[:,19:,:],dim="presentation_id"),dim="presentation_id")

In [ ]:
aligned_lfp_135
NaNs = 0
sus_channels = []
sus_presentations = []

for channel in range(len(aligned_lfp_315)):
    for presentation_id in range(channel):
        for time in range(presentation_id):
            if np.isnan(aligned_lfp_315[channel][presentation_id][time]):
                print("NaN at time {} in presentation {} in channel {}".format(time,presentation_id,channel))
                NaNs += 1
                if not (channel in sus_channels):
                    sus_channels.append(channel)
                if not (presentation_id in sus_presentations):
                    sus_presentations.append(presentation_id)
print(NaNs)
print("Sus channels:",sus_channels)
print("sus presentations:",sus_presentations)

[:8 + 12:14 + 19:]

In [ ]:
import scipy.stats as stats
import math

tStat, pValue = stats.ttest_ind(a= aligned_lfp_135,
               b = aligned_lfp_315,
               axis = 1,
               equal_var = False,
               nan_policy = 'raise')
print(tStat)
print(pValue.shape)

# MANUAL BENJAMINI-HOCHBERG

def benjamini_hochberg(p_values, alpha):
    indices_of_significant_values = []
    p_values_flat = p_values.flatten()
    dictionary = dict(zip(p_values_flat,range(len(p_values_flat))))
    
    sorted_keys = sorted(dictionary)
    print(sorted_keys[:100])
    
    greatest_i_so_far = 0

    for i in range(len(sorted_keys)):
        key = sorted_keys[i]
        critical_value = i*alpha/len(dictionary)
        if key<critical_value:
            greatest_i_so_far = i
    for i in range(greatest_i_so_far):
        indices_of_significant_values.append(dictionary.get(sorted_keys[i]))

    indices_of_significant_values = sorted(indices_of_significant_values)
    print("number of significant p-values:",len(indices_of_significant_values))
    
    return indices_of_significant_values

significant_indices = benjamini_hochberg(p_values = pValue, alpha = .05)

asdf = []
for i in range(pValue.shape[0]*pValue.shape[1]):
    if i in significant_indices:
        asdf.append(1)
    else:
        asdf.append(0)
print(len(asdf))
mask = []
for i in range(pValue.shape[0]):
    row = []
    for j in range(pValue.shape[1]):
        row.append(asdf[i*pValue.shape[1]+j])
    mask.append(row)

plt.figure(figsize=(8,6))
im = plt.imshow(mask, aspect='auto', origin='lower')
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')
        

In [ ]:
pValue.shape

In [ ]:
flashes_1_data = aligned_lfp_1.to_masked_array()
flashes_minus1_data = aligned_lfp_minus1.to_masked_array()
flashes_1_data.mask = np.ma.nomask
flashes_minus1_data.mask = np.ma.nomask
flashes_data_combined = np.concatenate((flashes_1_data,flashes_minus1_data),axis=1).transpose(1,0,2)
flashes_data_combined = flashes_data_combined.reshape(flashes_data_combined.shape[0],flashes_data_combined.shape[1] * flashes_data_combined.shape[2])
labels = []
for i in range(75):
    labels.append(1)
for i in range(75,150):
    labels.append(0)
print(flashes_data_combined.shape)
# print(labels)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(flashes_data_combined, labels, test_size=0.25, stratify=labels)
# print(x_train.shape)
# print(x_test.shape)
# print(len(y_train))
# print(len(y_test))
print(train_test_split)

In [ ]:
import sklearn.linear_model
model = sklearn.linear_model.LogisticRegression()
model.fit(x_train,y_train)
model.score(x_test,y_test)

In [ ]:
flashes_presentation_times

In [ ]:
flashes_presentation_table

In [ ]:
presentation_table_drifting_gratings = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'drifting_gratings] ']

In [ ]:
presentation_table_drifting_gratings

In [ ]:
drifting_gratings_orientations = presentation_table_drifting_gratings.orientation.unique()
#for orientation in drifting_gratings_orientations:

In [ ]:
presentation_table_drifting_gratings

presentation_table_drifting_gratings_315 = presentation_table_drifting_gratings[presentation_table_drifting_gratings.orientation == 315.0]
presentation_table_drifting_gratings_135 = presentation_table_drifting_gratings[presentation_table_drifting_gratings.orientation == 135.0]

In [ ]:
flashes_presentation_table

flaes_presentation_table_1 = flahses_presentation_table[flahses_presentation_table.color 
                                                                      == 1.0]
flahses_presentation_table_minus1 = flahses_presentation_table[flahses_presentation_table.color 
                                                                      == -1.0]

In [ ]:
flahses_presentation_table_1

In [ ]:
flahses_presentation_table_minus1

In [ ]:
print(session.stimulus_presentations.stimulus_name.unique())

In [ ]:
#CONSOLIDATION OF ALLEN CODE FOR PLOTTING LFP DATA ALIGNED TO START OF STIMULUS
def plot_aligned_LFP(lfp_data, presentation_table_master, stimulus_str, name_str):
    
    #get data and plot
    presentation_times = presentation_table_master.start_time.values
    presentation_ids = presentation_table_master.index.values

    trial_window = np.arange(-0.5, 0.5, 1/1000)
    time_selection = np.concatenate([trial_window + t for t in presentation_times])

    inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                      names=('presentation_id', 'time_from_presentation_onset'))

    ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
    ds = ds.assign(time=inds).unstack('time')

    aligned_lfp = ds['aligned_lfp']

    plt.figure(figsize=(8,6))
    plt.vlines(x = 500, ymin = 0, ymax = len(lfp_data.channel)-1, 
       colors = 'white', 
       label = 'vline_multiple - full height') 
    im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
    _ = plt.colorbar(im, fraction=0.036, pad=0.04)
    _ = plt.xlabel('Sample number')
    _ = plt.ylabel('Channel index')
    _ = plt.title('Aligned LFP for stimulus {}'.format(stimulus_str))
#     filename = '{}___stimulus___{}.png'.format(name_str,stimulus_str)
#     plt.imsave(os.path.join(out_directory, filename), im)

In [ ]:
 presentation_times = presentation_table_master.start_time.values

In [ ]:
#CUSTOM ATTEMPT TO PLOT ALIGNED LFPs UNDER DIFFERENT CONDITIONS
def plot_aligned_LFP_different_orientations(lfp_data, presentation_table_master, orientations, stimulus_str, name_str):
    for orientation in orientations:
        #make new DataArray slice
        presentation_table = presentation_table_master[presentation_table_master.orientation == orientation]
        
        #get data and plot
        presentation_times = presentation_table.start_time.values
        presentation_ids = presentation_table.index.values

        trial_window = np.arange(-0.5, 0.5, 1/1000)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp = ds['aligned_lfp']

        plt.figure(figsize=(8,6))
        plt.vlines(x = 500, ymin = 0, ymax = len(lfp_data.channel)-1, 
           colors = 'white', 
           label = 'vline_multiple - full height') 
        im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
        _ = plt.colorbar(im, fraction=0.036, pad=0.04)
        _ = plt.xlabel('Sample number')
        _ = plt.ylabel('Channel index')
        _ = plt.title('Aligned LFP for orientation {} for stimulus {}'.format(str(orientation), stimulus_str))
#         filename = '{}___orientation_{}___stimulus___{}.png'.format(name_str,str(orientation),stimulus_str)
#         plt.imsave(os.path.join(out_directory, filename), im)

def plot_aligned_LFP_different_colors(lfp_data, presentation_table_master, colors, stimulus_str, name_str):
    for color in colors:
        #make new DataArray slice
        presentation_table = presentation_table_master[presentation_table_master.color == color]
        
        #get data and plot
        presentation_times = presentation_table.start_time.values
        presentation_ids = presentation_table.index.values

        trial_window = np.arange(-0.5, 0.5, 1/1000)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp = ds['aligned_lfp']

        plt.figure(figsize=(8,6))
        plt.vlines(x = 500, ymin = 0, ymax = len(lfp_data.channel)-1, 
           colors = 'white', 
           label = 'vline_multiple - full height')
        im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
        _ = plt.colorbar(im, fraction=0.036, pad=0.04)
        _ = plt.xlabel('Sample number')
        _ = plt.ylabel('Channel index')
        _ = plt.title('Aligned LFP for color {} for stimulus {}'.format(str(color), stimulus_str))
#         filename = '{}___color_{}___stimulus___{}.png'.format(name_str,str(color),stimulus_str)
#         plt.imsave(os.path.join(out_directory, filename), im)
        
        


In [ ]:
print(len(lfp_VISp.channel))
print(presentation_table_drifting_gratings.orientation.unique())


CREATE PLOTS OF LFP DATA AVERAGED ACROSS TRIALS FOR DRIFTING GRATINGS, STATIC GRATINGS, AND FLASHES FOR DIFFERENT ORIENTATION/COLOR CONDITIONS WHERE RELEVANT AS WELL AS ACROSS ALL CONDITIONS

INDIVIDUAL CHANNELS DISPLAYED

In [ ]:
# presentation_table_drifting_gratings = session.stimulus_presentations[session.stimulus_presentations.stimulus_name 
#                                                                        == 'drifting_gratings']
# plot_aligned_LFP_different_orientations(lfp, presentation_table_drifting_gratings, 
#                                          presentation_table_drifting_gratings.orientation.unique(), 
#                                         'Drifting gratings', 'probeC_all')


# presentation_table_static_gratings = session.stimulus_presentations[session.stimulus_presentations.stimulus_name 
#                                                                     == 'static_gratings']
# plot_aligned_LFP_different_orientations(lfp, presentation_table_static_gratings, 
#                                        presentation_table_static_gratings.orientation.unique(), 
#                                       'Static gratings', 'probeC_all')

# plot_aligned_LFP_different_orientations(lfp_VISp, dg_presentation_table, dg_presentation_table.orientation.unique(),'Drifting gratings', 'probeC_VISp')
# presentation_table_drifting_gratings





# plot_aligned_LFP_different_orientations(lfp_VISp, presentation_table_static_gratings, 
#                                         presentation_table_static_gratings.orientation.unique(), 
#                                          'Static gratings', 'probeC_VISp')

presentation_table_flashes = session.stimulus_presentations[session.stimulus_presentations.stimulus_name 
                                                                     == 'flashes']
# plot_aligned_LFP_different_colors(lfp, presentation_table_flashes, 
#                                          presentation_table_flashes.color.unique(), 
#                                          'Flashes', 'probeC_all')

# plot_aligned_LFP_different_colors(lfp_VISp, presentation_table_flashes, 
#                                          presentation_table_flashes.color.unique(), 
#                                          'Flashes', 'probeC_VISp')

# plot_aligned_LFP(lfp, presentation_table_drifting_gratings, 'Drifting gratings', "probeC_all")

# plot_aligned_LFP(lfp, presentation_table_flashes, 'Flashes', "probeC_all")

# plot_aligned_LFP(lfp_VISp, presentation_table_drifting_gratings, 'Drifting gratings', "probeC_VISp")

# plot_aligned_LFP(lfp_VISp, presentation_table_flashes, 'Flashes', "probeC_VISp")


​
CREATE PLOTS OF LFP DATA AVERAGED ACROSS TRIALS FOR DRIFTING GRATINGS, STATIC GRATINGS, AND FLASHES FOR DIFFERENT ORIENTATION/COLOR CONDITIONS WHERE RELEVANT AS WELL AS ACROSS ALL CONDITIONS

CHANNELS ALSO AVERAGED

In [ ]:
presentation_table_drifting_gratings

Now, we can align the LFP to these presentation times using some xarray magic:

In [ ]:
trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

PLOTTING INDIVIDUAL TRIAL DATA FOR EACH COLOR OF FLASH STIMULUS

In [ ]:
presentation_table_flashes_minus1 = presentation_table_flashes[presentation_table_flashes.color == -1]
presentation_table_flashes_minus1

In [ ]:
presentation_table_flashes_plus1 = presentation_table_flashes[presentation_table_flashes.color == 1]
presentation_table_flashes_plus1

In [ ]:
flashes_presentation_table_1

In [ ]:
# print(aligned_lfp.shape)
# print(aligned_lfp.presentation_id)
# print(aligned_lfp[:,0,:])

presentation_times = flashes_presentation_table_1.start_time.values
presentation_ids = flashes_presentation_table_1.index.values
presentation_col = 1


# presentation_times = flashes_presentation_table_minus1.start_time.values
# presentation_ids = flashes_presentation_table_minus1.index.values
# presentation_col = -1

trial_window = np.arange(0.0, 0.5, 1/1000)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp_VISp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

# print(aligned_lfp)

for trial in range(len(aligned_lfp.presentation_id)):
    plt.figure(figsize=(8,6))
    im = plt.imshow(aligned_lfp[:,trial,:], aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
    _ = plt.colorbar(im, fraction=0.036, pad=0.04)
    _ = plt.xlabel('Sample number')
    _ = plt.ylabel('Channel index')
    _ = plt.title('Aligned LFPs to trial onset for flashes, trial {}'.format(str(aligned_lfp.presentation_id[trial])))

PLOTTING INDIVIDUAL TRIAL DATA FOR DRIFTING GRATING ORIENTATION 135 VS 315¶

In [ ]:
# print(aligned_lfp.shape)
# print(aligned_lfp.presentation_id)
# print(aligned_lfp[:,0,:])

# presentation_times = presentation_table_drifting_gratings_135.start_time.values
# presentation_ids = presentation_table_drifting_gratings_135.index.values
# presentation_or = 135
presentation_times = presentation_table_drifting_gratings_315.start_time.values
presentation_ids = presentation_table_drifting_gratings_315.index.values
presentation_or = 315

trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

for trial in range(len(aligned_lfp.presentation_id)):
    plt.figure(figsize=(8,6))
    im = plt.imshow(aligned_lfp[-20:-10,trial,:], aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
    _ = plt.colorbar(im, fraction=0.036, pad=0.04)
    _ = plt.xlabel('Sample number')
    _ = plt.ylabel('Channel index')
    _ = plt.title('Aligned LFPs to trial onset for drifting gratings, trial {}'.format(str(aligned_lfp.presentation_id[trial])))

## Probe D

The motivation in investigating Probe D is to isolate the LGD (lateral Geniculate Nucleus, LGN) as it may reecieve the stimulation prior to V1

### NWBWIDGETS

In [ ]:
from pynwb import NWBHDF5IO
import allensdk.brain_observatory.ecephys.nwb

from nwbwidgets import nwb2widget
fpath = '/Users/willi/downloads/session_715093703/probe_810755803_lfp.nwb'
nwb = NWBHDF5IO(fpath, 'r').read()

#below works if you are analyzing the session file as input rather than a probe. remmeber the .nwb ext
#from nwbwidgets.allen import AllenRasterWidget

#AllenRasterWidget(nwb.units)


In [ ]:
nwb2widget(nwb)


## LFP Exploration

##### Below is loading in the specific probe of interest and selecting the region of interest LGd

In [ ]:
probe_id = session.probes[session.probes.description == 'probeD'].index.values[0]
lfp = session.get_lfp(probe_id)
lfp_LGd = lfp.sel(channel = lfp.channel[0:21])


In [ ]:
session.probes


In [ ]:
lfp

In [ ]:
lfp_LGd

Next we desire to isolate time slices and visualize them through a plot

In [ ]:
lfp_slice = lfp.sel(time=slice(1800,5410))

lfp_slice_LGd = lfp_slice.sel(channel=lfp_slice.channel[62:84])


In [ ]:
lfp_slice = lfp.sel(time=slice(1800,5410))

lfp_slice_LGd = lfp_slice.sel(channel=lfp_slice.channel[-25:-10]) ##playing around with the range for the matplotlib...
#...function further down


Obtaining multiple plots at once for lfp_slice_LGd with a for loop

In [ ]:
for i in range(15):
    plt.figure(figsize=(10,2))
    _ = plt.plot(lfp_slice_LGd.time, lfp_slice_LGd.sel(channel=lfp_slice_LGd.channel[i]))
    plt.xlabel('Time (s)')
    plt.ylabel('LFP (V)')

In [ ]:
lfp_slice_LGd

In [ ]:
lfp.sel(channel=lfp.channel[62:84])

In [ ]:
plt.figure(figsize=(10,2))
_ = plt.plot(lfp_slice_LGd.time, lfp_slice_LGd)
plt.xlabel('Time (s)')
plt.ylabel('LFP (V)')

###### Start of LGd Analysis for Flashes -1 and +1

Yes, this code generates the desired graphs, you just have to do them one at a time changing out the time and id for the 1 and -1 conditions.

In [ ]:
LGd= lfp.sel(channel=lfp.channel[0:21])

In [ ]:
LGdFlashes=LGd.sel(time=slice(1800,3600))

In [ ]:
plt.figure(figsize=(10,2))
_ = plt.plot(LGdFlashes.time, LGdFlashes)
plt.xlabel('Time (s)')
plt.ylabel('LFP (V)')

In [ ]:
session.stimulus_presentations

In [ ]:

flashes_presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'flashes']


In [ ]:
flashes_presentation_table_plus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                      == 1.0]
flashes_presentation_table_minus1 = flashes_presentation_table[flashes_presentation_table.color 
                                                                      == -1.0]

In [ ]:
flashes_presentation_table_minus1

In [ ]:
flashes_presentation_table_plus1

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'flashes']

presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values

In [ ]:
presentation_table_plus1 = presentation_table[presentation_table.color==1]
presentation_times_plus1 = presentation_table_plus1.start_time.values
presentation_ids_plus1 = presentation_table_plus1.index.values

In [ ]:
presentation_table_minus1 = presentation_table[presentation_table.color==-1]
presentation_times_minus1 = presentation_table_minus1.start_time.values
presentation_ids_minus1 = presentation_table_minus1.index.values

In [ ]:
trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times_plus1])

inds = pd.MultiIndex.from_product((presentation_ids_plus1, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_plus1 = ds['aligned_lfp']

In [ ]:
plt.figure(figsize=(8,6))
im = plt.imshow(aligned_lfp_plus1.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')

In [ ]:
trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times_minus1])

inds = pd.MultiIndex.from_product((presentation_ids_minus1, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp_minus1 = ds['aligned_lfp']

In [ ]:
aligned_lfp_minus1

In [ ]:
plt.figure(figsize=(8,6))
im = plt.imshow(aligned_lfp_minus1.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')

In [ ]:
# PLOTTING ABSOLUTE DIFFERENCE BETWEEN THE TWO

plt.figure(figsize=(8,6))
im = plt.imshow(aligned_lfp_plus1.mean(dim='presentation_id') - aligned_lfp_minus1.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')



In [ ]:
difference = aligned_lfp_plus1 - aligned_lfp_minus1
difference
# difference_mean = aligned_lfp_plus1.mean(dim='presentation_id') - aligned_lfp_minus1.mean(dim='presentation_id')
# difference_mean

T test for differences Null Hypothesis H0: There is no statistically significant difference between the means of the two stimulus conditions acoss time points. Alternative Hypothesis HA: There is a statistically significant difference between the means of the two stimulus conditions across time points.

In [ ]:
import scipy.stats as stats
import math

In [ ]:


tStat, pValue = stats.ttest_ind(a= aligned_lfp_minus1,
               b = aligned_lfp_plus1,
               axis = 1,
               equal_var = False)

In [ ]:
print(aligned_lfp_plus1.shape)

In [ ]:
tstat = tTest.statistic

In [ ]:
tstat

In [ ]:
print("P-Value:{} T-Statistic:{}".format(pValue,tStat))

In [ ]:
len(pValue[0])
p_values_flat = pValue.flatten()
p_values_flat = p_values_flat.tolist()
indices = range(len(p_values_flat))
print(indices)
p_index_dictionary = dict(zip(p_values_flat,indices))
sorted_keys = sorted(p_index_dictionary)
print((sorted_keys[2500]))
# print(p_index_dictionary)
# print(len(p_values_flat))
# print(p_index_dictionary)
# print(sorted(p_index_dictionary))
# print(len(p_index_dictionary.keys()))
# print(p_index_dictionary.items())
# print(list(p_index_dictionary.keys())[0])

In [ ]:
# MANUAL BENJAMINI-HOCHBERG

def benjamini_hochberg(p_values, alpha):
    indices_of_significant_values = []
    p_values_flat = p_values.flatten()
    dictionary = dict(zip(p_values_flat,range(len(p_values_flat))))
    
    sorted_keys = sorted(dictionary)
    
    greatest_i_so_far = 0

    for i in range(len(sorted_keys)):
        key = sorted_keys[i]
        critical_value = i*alpha/len(dictionary)
        if key<critical_value:
            greatest_i_so_far = i
    for i in range(greatest_i_so_far):
        indices_of_significant_values.append(dictionary.get(sorted_keys[i]))

    indices_of_significant_values = sorted(indices_of_significant_values)
    print("number of significant p-values:",len(indices_of_significant_values))
    
    return indices_of_significant_values

significant_indices = benjamini_hochberg(p_values = pValue, alpha = .00000000000005)

In [ ]:

asdf = []
for i in range(pValue.shape[0]*pValue.shape[1]):
    if i in significant_indices:
        asdf.append(1)
    else:
        asdf.append(0)
print(len(asdf))
mask = []
for i in range(pValue.shape[0]):
    row = []
    for j in range(pValue.shape[1]):
        row.append(asdf[i*pValue.shape[1]+j])
    mask.append(row)

plt.figure(figsize=(8,6))
im = plt.imshow(mask, aspect='auto', origin='lower')
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')
        

In [ ]:
pValue.shape[0]

In [ ]:
plt.figure(figsize=(8,6))
im = plt.imshow(tStat, aspect='auto', origin='lower')
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')

In [ ]:
plt.figure(figsize=(8,6))
im = plt.imshow(np.log(pValue), aspect='auto', origin='lower')
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')

In [ ]:
ls = tStat.tolist()

In [ ]:
tStat.shape

In [ ]:
import chart_studio.plotly as cs
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

In [ ]:
cs.iplot(ls, filename='normal-dists-plot')

In [ ]:
plot_aligned_LFP_different_colors(lfp, presentation_table_flashes, 
                                     presentation_table_flashes.color.unique(), 
                                   'Flashes', 'probeD_all')

In [ ]:
plot_aligned_LFP_different_colors(lfp_LGd, presentation_table_flashes,                                         
                                  presentation_table_flashes.color.unique(), 
                                     'Flashes', 'probeD_LGd')

In [ ]:
plot_aligned_LFP(lfp, presentation_table_flashes, 'Flashes', "probeD_all")


In [ ]:
plot_aligned_LFP(lfp_LGd, presentation_table_flashes, 'Flashes', "probeD_LGd")

In [ ]:
# print(aligned_lfp.shape)
# print(aligned_lfp.presentation_id)
# print(aligned_lfp[:,0,:])

# presentation_times = presentation_table_flashes_1.start_time.values
# presentation_ids = presentation_table_flashes_1.index.values
# presentation_col = 1
presentation_times = presentation_table_flashes_minus1.start_time.values
presentation_ids = presentation_table_flashes_minus1.index.values
presentation_col = -1

trial_window = np.arange(-0.5, 0.5, 1/1000)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

for trial in range(5):    #(len(aligned_lfp.presentation_id)):
    plt.figure(figsize=(8,6))
    im = plt.imshow(aligned_lfp[-25:,trial,:], aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
    _ = plt.colorbar(im, fraction=0.036, pad=0.04)
    _ = plt.xlabel('Sample number')
    _ = plt.ylabel('Channel index')
    _ = plt.title('Aligned LFPs to trial onset for flashes, trial {}'.format(str(aligned_lfp.presentation_id[trial])))

In [ ]:
presentation_table_flashes

In [ ]:
lfp.mean()

In [ ]:
lfp_LGd.mean()

###### End of LGd Analysis for Flashes -1 and +1

In [ ]:
plt.figure(figsize=(16,16))
im = plt.imshow(lfp_slice_LGd.T,aspect='auto',origin='lower',vmin=-1e-3, vmax=1e-3)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')

Note that we've transposed the original array to place the time dimension along the x-axis. We've also configured the plot so that the origin of the array is in the lower-left, so that that channels closer to the probe tip are lower in the image.

A few things to note about this plot:

The units of the LFP are volts, so the color scale ranges from -1 to +1 mV

Even though there are 384 channels on the Neuropixels probe, there are only 95 channels in this plot. That's because only every 4th channel is included in the NWB file (resulting in 40 micron vertical spacing). In addition, the reference channels and channels far outside the brain have been removed.

The top of the plot is relatively flat. This corresponds to channels that are outside the brain. The LFP channels are originally referenced to a ground wire embedded in the ACSF/agarose mixture about cortex. Before NWB packaging, the LFP data is digitally referenced to the channels outside the brain, to remove noise that's shared across the whole probe. There's a large increase in LFP power toward the middle of the probe, which corresponds to channels in hippocampus.

Let's do some additional data selection to look at just the visual recordings

In [ ]:
channel_ids = session.channels[(session.channels.probe_id == probe_id) & \
                 (session.channels.ecephys_structure_acronym.isin(['LGd']))].index.values

lfp_slice2 = lfp_slice.sel(channel=slice(np.min(channel_ids), np.max(channel_ids)))

plt.figure(figsize=(8,4))
im = plt.imshow(lfp_slice2.T,aspect='auto',origin='lower',vmin=-1e-3, vmax=1e-3)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number') ## Sample number is time: think of it like each time point is a sample
_ = plt.ylabel('Channel index')

What follows next is a sequence of code cells from Written by Clay, edited by myself which appear to be attempts to isolate the stimulus presentation for the probe. The code is fairly straightforward and will be used for work on Probe D Analysis

In [ ]:

flahses_presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'flashes']

flahses_presentation_times = flahses_presentation_table.start_time.values
flahses_presentation_ids = flahses_presentation_table.index.values


In [ ]:
flahses_presentation_table


In [ ]:
presentation_table_drifting_gratings = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'drifting_gratings] ']

In [ ]:
presentation_table_drifting_gratings

In [ ]:
drifting_gratings_orientations = presentation_table_drifting_gratings.orientation.unique()
#for orientation in drifting_gratings_orientations:

In [ ]:
flahses_presentation_table

flahses_presentation_table_1 = flahses_presentation_table[flahses_presentation_table.color 
                                                                      == 1.0]
flahses_presentation_table_minus1 = flahses_presentation_table[flahses_presentation_table.color 
                                                                      == -1.0]

In [ ]:
flahses_presentation_table_1

In [ ]:
flahses_presentation_table_minus1

In [ ]:
print(session.stimulus_presentations.stimulus_name.unique())

In [ ]:
#CONSOLIDATION OF ALLEN CODE FOR PLOTTING LFP DATA ALIGNED TO START OF STIMULUS
def plot_aligned_LFP(lfp_data, presentation_table_master, stimulus_str, name_str):
    
    #get data and plot
    presentation_times = presentation_table_master.start_time.values
    presentation_ids = presentation_table_master.index.values

    trial_window = np.arange(-0.5, 0.5, 1/1000)
    time_selection = np.concatenate([trial_window + t for t in presentation_times])

    inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                      names=('presentation_id', 'time_from_presentation_onset'))

    ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
    ds = ds.assign(time=inds).unstack('time')

    aligned_lfp = ds['aligned_lfp']

    plt.figure(figsize=(8,6))
    plt.vlines(x = 500, ymin = 0, ymax = len(lfp_data.channel)-1, 
       colors = 'white', 
       label = 'vline_multiple - full height') 
    im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
    _ = plt.colorbar(im, fraction=0.036, pad=0.04)
    _ = plt.xlabel('Sample number')
    _ = plt.ylabel('Channel index')
    _ = plt.title('Aligned LFP for stimulus {}'.format(stimulus_str))
#     filename = '{}___stimulus___{}.png'.format(name_str,stimulus_str)
#     plt.imsave(os.path.join(out_directory, filename), im)

In [ ]:
#CUSTOM ATTEMPT TO PLOT ALIGNED LFPs UNDER DIFFERENT CONDITIONS
def plot_aligned_LFP_different_orientations(lfp_data, presentation_table_master, orientations, stimulus_str, name_str):
    for orientation in orientations:
        #make new DataArray slice
        presentation_table = presentation_table_master[presentation_table_master.orientation == orientation]
        
        #get data and plot
        presentation_times = presentation_table.start_time.values
        presentation_ids = presentation_table.index.values

        trial_window = np.arange(-0.5, 0.5, 1/1000)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp = ds['aligned_lfp']

        plt.figure(figsize=(8,6))
        plt.vlines(x = 500, ymin = 0, ymax = len(lfp_data.channel)-1, 
           colors = 'white', 
           label = 'vline_multiple - full height') 
        im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
        _ = plt.colorbar(im, fraction=0.036, pad=0.04)
        _ = plt.xlabel('Sample number')
        _ = plt.ylabel('Channel index')
        _ = plt.title('Aligned LFP for orientation {} for stimulus {}'.format(str(orientation), stimulus_str))
#         filename = '{}___orientation_{}___stimulus___{}.png'.format(name_str,str(orientation),stimulus_str)
#         plt.imsave(os.path.join(out_directory, filename), im)

def plot_aligned_LFP_different_colors(lfp_data, presentation_table_master, colors, stimulus_str, name_str):
    for color in colors:
        #make new DataArray slice
        presentation_table = presentation_table_master[presentation_table_master.color == color]
        
        #get data and plot
        presentation_times = presentation_table.start_time.values
        presentation_ids = presentation_table.index.values

        trial_window = np.arange(-0.5, 0.5, 1/1000)
        time_selection = np.concatenate([trial_window + t for t in presentation_times])

        inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                          names=('presentation_id', 'time_from_presentation_onset'))

        ds = lfp_data.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
        ds = ds.assign(time=inds).unstack('time')

        aligned_lfp = ds['aligned_lfp']

        plt.figure(figsize=(8,6))
        plt.vlines(x = 500, ymin = 0, ymax = len(lfp_data.channel)-1, 
           colors = 'white', 
           label = 'vline_multiple - full height')
        im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
        _ = plt.colorbar(im, fraction=0.036, pad=0.04)
        _ = plt.xlabel('Sample number')
        _ = plt.ylabel('Channel index')
        _ = plt.title('Aligned LFP for color {} for stimulus {}'.format(str(color), stimulus_str))
#         filename = '{}___color_{}___stimulus___{}.png'.format(name_str,str(color),stimulus_str)
#         plt.imsave(os.path.join(out_directory, filename), im)
        
        


In [ ]:
print(len(lfp_LGd.channel))
print(presentation_table_drifting_gratings.orientation.unique())


CREATE PLOTS OF LFP DATA AVERAGED ACROSS TRIALS FOR DRIFTING GRATINGS, STATIC GRATINGS, AND FLASHES FOR DIFFERENT ORIENTATION/COLOR CONDITIONS WHERE RELEVANT AS WELL AS ACROSS ALL CONDITIONS

INDIVIDUAL CHANNELS DISPLAYED

In [ ]:
presentation_table_drifting_gratings = session.stimulus_presentations[session.stimulus_presentations.stimulus_name 
                                                                       == 'drifting_gratings']
plot_aligned_LFP_different_orientations(lfp, presentation_table_drifting_gratings, 
                                         presentation_table_drifting_gratings.orientation.unique(), 
                                         'Drifting gratings', 'probeD_all')


presentation_table_static_gratings = session.stimulus_presentations[session.stimulus_presentations.stimulus_name 
                                                                     == 'static_gratings']
plot_aligned_LFP_different_orientations(lfp, presentation_table_static_gratings, 
                                         presentation_table_static_gratings.orientation.unique(), 
                                         'Static gratings', 'probeD_all')


plot_aligned_LFP_different_orientations(lfp_LGd, presentation_table_drifting_gratings, 
                                         presentation_table_drifting_gratings.orientation.unique(), 
                                         'Drifting gratings', 'probeD_LGd')

plot_aligned_LFP_different_orientations(lfp_LGd, presentation_table_static_gratings, 
                                         presentation_table_static_gratings.orientation.unique(), 
                                         'Static gratings', 'probeD_LGd')


presentation_table_flashes = session.stimulus_presentations[session.stimulus_presentations.stimulus_name 
                                                                    == 'flashes']
plot_aligned_LFP_different_colors(lfp, presentation_table_flashes, 
                                     presentation_table_flashes.color.unique(), 
                                   'Flashes', 'probeD_all')

plot_aligned_LFP_different_colors(lfp_LGd, presentation_table_flashes,                                         
                                  presentation_table_flashes.color.unique(), 
                                     'Flashes', 'probeD_LGd')

plot_aligned_LFP(lfp, presentation_table_drifting_gratings, 'Drifting gratings', "probeD_all")

plot_aligned_LFP(lfp, presentation_table_flashes, 'Flashes', "probeD_all")

plot_aligned_LFP(lfp_LGd, presentation_table_drifting_gratings, 'Drifting gratings', "probeD_LGd")

plot_aligned_LFP(lfp_LGd, presentation_table_flashes, 'Flashes', "probeD_LGd")




In [ ]:
#plot_aligned_LFP_different_colors(lfp_LGd, presentation_table_flashes,                                         
                                  presentation_table_flashes.color.unique(), 
                                     'Flashes', 'probeD_LGd')

In [ ]:
plot_aligned_LFP_different_colors(lfp, presentation_table_flashes, 
                                     presentation_table_flashes.color.unique(), 
                                   'Flashes', 'probeD_all')


In [ ]:
plot_aligned_LFP(lfp, presentation_table_flashes, 'Flashes', "probeD_all")

In [ ]:
plot_aligned_LFP(lfp_LGd, presentation_table_flashes, 'Flashes', "probeD_LGd")


In [ ]:
presentation_table_flashes_minus1 = presentation_table_flashes[presentation_table_flashes.color == -1]
presentation_table_flashes_minus1

In [ ]:
presentation_table_flashes_plus1 = presentation_table_flashes[presentation_table_flashes.color == 1]
presentation_table_flashes_plus1

In [ ]:
lfp


In [ ]:
import cv2
 
# read image as grey scale
ProbeD_drifting_LGd = cv2.imread('/User/Willi/Downloads', cv2.ProbeDDriftingGratingsLGD)
 
# save image
status = cv2.imwrite('/User/Willi/Downloads', cv2.ProbeDDriftingGratingsLGD)
 
print("Image written to file-system : ",status)

​CREATE PLOTS OF LFP DATA AVERAGED ACROSS TRIALS FOR DRIFTING GRATINGS, STATIC GRATINGS, AND FLASHES FOR DIFFERENT ORIENTATION/COLOR CONDITIONS WHERE RELEVANT AS WELL AS ACROSS ALL CONDITIONS

CHANNELS ALSO AVERAGED

In [ ]:
presentation_table_drifting_gratings

In [ ]:
trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

PLOTTING INDIVIDUAL TRIAL DATA FOR EACH COLOR OF FLASH STIMULUS

In [ ]:
# print(aligned_lfp.shape)
# print(aligned_lfp.presentation_id)
# print(aligned_lfp[:,0,:])

# presentation_times = presentation_table_flashes_1.start_time.values
# presentation_ids = presentation_table_flashes_1.index.values
# presentation_col = 1
presentation_times = presentation_table_flashes_minus1.start_time.values
presentation_ids = presentation_table_flashes_minus1.index.values
presentation_col = -1

trial_window = np.arange(-0.5, 0.5, 1/1000)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

for trial in range(5):    #(len(aligned_lfp.presentation_id)):
    plt.figure(figsize=(8,6))
    im = plt.imshow(aligned_lfp[-25:,trial,:], aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
    _ = plt.colorbar(im, fraction=0.036, pad=0.04)
    _ = plt.xlabel('Sample number')
    _ = plt.ylabel('Channel index')
    _ = plt.title('Aligned LFPs to trial onset for flashes, trial {}'.format(str(aligned_lfp.presentation_id[trial])))

In [ ]:
ds

PLOTTING INDIVIDUAL TRIAL DATA FOR DRIFTING GRATING ORIENTATION 135 VS 315¶

In [ ]:
# print(aligned_lfp.shape)
# print(aligned_lfp.presentation_id)
# print(aligned_lfp[:,0,:])

# presentation_times = presentation_table_drifting_gratings_135.start_time.values
# presentation_ids = presentation_table_drifting_gratings_135.index.values
 presentation_or = 135
presentation_times = presentation_table_drifting_gratings_315.start_time.values
presentation_ids = presentation_table_drifting_gratings_315.index.values
presentation_or = 315

trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

for trial in range(len(aligned_lfp.presentation_id)):
    plt.figure(figsize=(8,6))
    im = plt.imshow(aligned_lfp[-20:-10,trial,:], aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
    _ = plt.colorbar(im, fraction=0.036, pad=0.04)
    _ = plt.xlabel('Sample number')
    _ = plt.ylabel('Channel index')
    _ = plt.title('Aligned LFPs to trial onset for drifting gratings, trial {}'.format(str(aligned_lfp.presentation_id[trial])))